<a href="https://colab.research.google.com/github/nmansour67/skills-introduction-to-github/blob/main/Oman_KSA_Trade_Analysis_ReportDec8NYC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================================================================
# CELL 1: SETUP & INSTALLATION
# ============================================================================
print("="*80)
print("COMPREHENSIVE BILATERAL TRADE ANALYSIS - ALL DATA SOURCES")
print("="*80)

import sys
import subprocess

packages = ['python-docx', 'openpyxl', 'scikit-learn', 'scipy', 'pandas', 'numpy']
print("\n📦 Installing packages...")
for package in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from datetime import datetime
from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Setup complete!")

COMPREHENSIVE BILATERAL TRADE ANALYSIS - ALL DATA SOURCES

📦 Installing packages...
✅ Setup complete!


In [21]:
# ============================================================================
# CELL 2: UPLOAD ALL DATA FILES
# ============================================================================
from google.colab import files
import os

print("="*80)
print("📁 UPLOAD ALL YOUR DATA FILES")
print("="*80)
print("\nPlease upload these 4 files:")
print("  1. Trade_Balance_Oman_KSA.csv")
print("  2. Oman_Export_to_KSA_HS6_10Yrs_Value.csv")
print("  3. Oman_Import_from_KSA_HS6_10Yrs_Value.csv")
print("  4. Oman_Re-export_to_KSA_HS6_10Yrs_Value.csv")
print("\n(Select all 4 files at once)\n")

uploaded = files.upload()

print("\n" + "="*80)
print("📦 FILES UPLOADED:")
print("="*80)
for filename, content in uploaded.items():
    print(f"  ✓ {filename} ({len(content):,} bytes)")

# Identify files
balance_file = None
export_file = None
import_file = None
reexport_file = None

for filename in uploaded.keys():
    if 'balance' in filename.lower():
        balance_file = filename
    elif 'export' in filename.lower() and 're-export' not in filename.lower():
        export_file = filename
    elif 'import' in filename.lower():
        import_file = filename
    elif 're-export' in filename.lower():
        reexport_file = filename

print("\n📋 FILE MAPPING:")
print(f"  Balance:    {balance_file}")
print(f"  Exports:    {export_file}")
print(f"  Imports:    {import_file}")
print(f"  Re-exports: {reexport_file}")
print("="*80)

📁 UPLOAD ALL YOUR DATA FILES

Please upload these 4 files:
  1. Trade_Balance_Oman_KSA.csv
  2. Oman_Export_to_KSA_HS6_10Yrs_Value.csv
  3. Oman_Import_from_KSA_HS6_10Yrs_Value.csv
  4. Oman_Re-export_to_KSA_HS6_10Yrs_Value.csv

(Select all 4 files at once)



Saving Oman_Export_to_KSA_HS6_10Yrs_Value.csv to Oman_Export_to_KSA_HS6_10Yrs_Value.csv
Saving Oman_Import_from_KSA_HS6_10Yrs_Value.csv to Oman_Import_from_KSA_HS6_10Yrs_Value.csv
Saving Oman_Re-export_to_KSA_HS6_10Yrs_Value.csv to Oman_Re-export_to_KSA_HS6_10Yrs_Value.csv
Saving Trade_Balance_Oman_KSA.csv to Trade_Balance_Oman_KSA (1).csv

📦 FILES UPLOADED:
  ✓ Oman_Export_to_KSA_HS6_10Yrs_Value.csv (3,556 bytes)
  ✓ Oman_Import_from_KSA_HS6_10Yrs_Value.csv (3,558 bytes)
  ✓ Oman_Re-export_to_KSA_HS6_10Yrs_Value.csv (3,064 bytes)
  ✓ Trade_Balance_Oman_KSA (1).csv (853 bytes)

📋 FILE MAPPING:
  Balance:    Trade_Balance_Oman_KSA (1).csv
  Exports:    Oman_Export_to_KSA_HS6_10Yrs_Value.csv
  Imports:    Oman_Import_from_KSA_HS6_10Yrs_Value.csv
  Re-exports: Oman_Re-export_to_KSA_HS6_10Yrs_Value.csv


In [14]:
# ============================================================================
# CELL 3: CONVERT .NUMBERS FILES TO CSV
# ============================================================================

print("\n🔄 Converting .numbers files to CSV format...")
print("="*80)

# Install numbers-parser
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numbers-parser"])

from numbers_parser import Document as NumbersDocument

def convert_numbers_to_dataframe(filename):
    """Convert .numbers file to pandas DataFrame"""
    try:
        doc = NumbersDocument(filename)
        sheets = doc.sheets

        if len(sheets) == 0:
            print(f"  ⚠️  No sheets found in {filename}")
            return None

        # Get first sheet and first table
        sheet = sheets[0]
        tables = sheet.tables

        if len(tables) == 0:
            print(f"  ⚠️  No tables found in {filename}")
            return None

        table = tables[0]

        # Extract data
        data = []
        num_rows = table.num_rows
        num_cols = table.num_cols

        for row in range(num_rows):
            row_data = []
            for col in range(num_cols):
                cell = table.cell(row, col)
                row_data.append(cell.value if cell.value is not None else '')
            data.append(row_data)

        # Create DataFrame
        df = pd.DataFrame(data)

        # Try to set first row as header
        if len(df) > 0:
            df.columns = df.iloc[0]
            df = df[1:].reset_index(drop=True)

        return df

    except Exception as e:
        print(f"  ❌ Error converting {filename}: {str(e)}")
        return None

# Convert export file
if export_file:
    print(f"\n📊 Processing: {export_file}")
    df_exports = convert_numbers_to_dataframe(export_file)
    if df_exports is not None:
        print(f"  ✓ Shape: {df_exports.shape}")
        print(f"  ✓ Columns: {list(df_exports.columns[:5])}...")
        df_exports.to_csv('/content/exports_converted.csv', index=False)
    else:
        df_exports = pd.DataFrame()

# Convert import file
if import_file:
    print(f"\n📊 Processing: {import_file}")
    df_imports = convert_numbers_to_dataframe(import_file)
    if df_imports is not None:
        print(f"  ✓ Shape: {df_imports.shape}")
        print(f"  ✓ Columns: {list(df_imports.columns[:5])}...")
        df_imports.to_csv('/content/imports_converted.csv', index=False)
    else:
        df_imports = pd.DataFrame()

# Convert re-export file
if reexport_file:
    print(f"\n📊 Processing: {reexport_file}")
    df_reexports = convert_numbers_to_dataframe(reexport_file)
    if df_reexports is not None:
        print(f"  ✓ Shape: {df_reexports.shape}")
        print(f"  ✓ Columns: {list(df_reexports.columns[:5])}...")
        df_reexports.to_csv('/content/reexports_converted.csv', index=False)
    else:
        df_reexports = pd.DataFrame()

print("\n✅ Conversion complete!")
print("="*80)


🔄 Converting .numbers files to CSV format...

📊 Processing: Oman_Export_to_KSA_HS6_10Yrs_Value.numbers
  ❌ Error converting Oman_Export_to_KSA_HS6_10Yrs_Value.numbers: invalid Numbers document (missing files)

📊 Processing: Oman_Import_from_KSA_HS6_10Yrs_Value.numbers
  ✓ Shape: (21, 12)
  ✓ Columns: ['Product code', 'Product label', np.float64(2015.0), np.float64(2016.0), np.float64(2017.0)]...

📊 Processing: Oman_Re-export_to_KSA_HS6_10Yrs_Value.numbers
  ✓ Shape: (21, 12)
  ✓ Columns: ['Product code', 'Product label', np.float64(2014.0), np.float64(2015.0), np.float64(2016.0)]...

✅ Conversion complete!


In [30]:
# ============================================================================
# CELL 4: LOAD TRADE BALANCE DATA (FIXED IMPORT DETECTION)
# ============================================================================

from io import StringIO

def clean_value(val):
    """Clean currency values"""
    if pd.isna(val) or val == '':
        return 0.0
    if isinstance(val, (int, float)):
        return float(val)
    val_str = str(val).replace('(', '-').replace(')', '').replace('$', '').replace(',', '').strip()
    try:
        return float(val_str)
    except:
        return 0.0

print("\n📊 LOADING TRADE BALANCE DATA")
print("="*80)

if not balance_file:
    print("❌ No balance file found!")
else:
    print(f"Loading: {balance_file}")

    # Load CSV
    df = pd.read_csv(StringIO(uploaded[balance_file].decode('utf-8')))

    print(f"  Loaded: {len(df)} rows × {len(df.columns)} columns")
    print(f"  Columns: {list(df.columns)}")

    # Show what's in the first column (row labels)
    print(f"\n  Row labels in file:")
    for idx, row in df.iterrows():
        print(f"    • {row.iloc[0]}")

    # Extract year columns
    year_cols = []
    for col in df.columns:
        col_str = str(col).strip()
        if col_str.startswith('20') and len(col_str) == 4:
            try:
                year = int(col_str)
                if 2010 <= year <= 2030:
                    year_cols.append(col)
            except:
                pass

    print(f"\n  ✅ Years detected: {', '.join([str(y) for y in year_cols])}")

    # Extract rows - FIXED PATTERNS
    print(f"\n  Detecting trade components...")

    # Pattern 1: Exports (not re-exports)
    exports_row = df[
        df.iloc[:, 0].astype(str).str.contains('export', case=False, na=False) &
        ~df.iloc[:, 0].astype(str).str.contains('re-export', case=False, na=False)
    ]

    # Pattern 2: Re-exports
    reexports_row = df[
        df.iloc[:, 0].astype(str).str.contains('re-export', case=False, na=False)
    ]

    # Pattern 3: Imports - FIXED TO MATCH YOUR FORMAT
    # Your file has "Imports Oman to KSA" not "Imports Oman from KSA"
    imports_row = df[
        df.iloc[:, 0].astype(str).str.contains('import', case=False, na=False)
    ]

    print(f"    Exports row: {'✅ Found' if len(exports_row) > 0 else '❌ Not found'}")
    print(f"    Re-exports row: {'✅ Found' if len(reexports_row) > 0 else '❌ Not found'}")
    print(f"    Imports row: {'✅ Found' if len(imports_row) > 0 else '❌ Not found'}")

    # Show what we found
    if len(exports_row) > 0:
        print(f"      Export row label: '{exports_row.iloc[0, 0]}'")
    if len(reexports_row) > 0:
        print(f"      Re-export row label: '{reexports_row.iloc[0, 0]}'")
    if len(imports_row) > 0:
        print(f"      Import row label: '{imports_row.iloc[0, 0]}'")

    # Build trade dataframe
    trade_data = {}
    for year in year_cols:
        trade_data[str(year)] = {
            'Exports': clean_value(exports_row[year].iloc[0] if len(exports_row) > 0 else 0),
            'Re-exports': clean_value(reexports_row[year].iloc[0] if len(reexports_row) > 0 else 0),
            'Imports': clean_value(imports_row[year].iloc[0] if len(imports_row) > 0 else 0)
        }

    df_trade = pd.DataFrame(trade_data).T
    df_trade.index.name = 'Year'

    # Calculate derived metrics
    df_trade['Total_Exports'] = df_trade['Exports'] + df_trade['Re-exports']
    df_trade['Trade_Balance'] = df_trade['Total_Exports'] - df_trade['Imports']

    # Fix Coverage Ratio calculation (avoid division by zero)
    df_trade['Coverage_Ratio'] = df_trade.apply(
        lambda row: (row['Total_Exports'] / row['Imports'] * 100) if row['Imports'] > 0 else 0,
        axis=1
    ).round(1)

    df_trade['Trade_Volume'] = df_trade['Total_Exports'] + df_trade['Imports']

    print(f"\n✅ TRADE BALANCE DATA LOADED!")
    print("="*80)
    print("\n📋 ANNUAL TRADE DATA:")
    print(df_trade)
    print("\n" + "="*80)

    # Verification - check if imports are actually loaded
    total_imports = df_trade['Imports'].sum()
    if total_imports == 0:
        print("\n⚠️ WARNING: All import values are ZERO!")
        print("   This indicates a data loading issue.")
        print("\n   Showing raw data from CSV:")
        print(df)
    else:
        print(f"\n✅ Import data verified: Total imports = ${total_imports/1e9:.2f}B")

print("="*80)


📊 LOADING TRADE BALANCE DATA
Loading: Trade_Balance_Oman_KSA (1).csv
  Loaded: 5 rows × 11 columns
  Columns: ['Product code', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']

  Row labels in file:
    • Exports Oman to KSA $
    • Re-exports Oman to KSA $
    • Imports Oman to KSA $
    • Trade Balance without Re-Export
    • Trade Balance with Re-Export

  ✅ Years detected: 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024

  Detecting trade components...
    Exports row: ✅ Found
    Re-exports row: ✅ Found
    Imports row: ✅ Found
      Export row label: 'Exports Oman to KSA $'
      Re-export row label: 'Re-exports Oman to KSA $'
      Import row label: 'Imports Oman to KSA $'

✅ TRADE BALANCE DATA LOADED!

📋 ANNUAL TRADE DATA:
           Exports   Re-exports       Imports  Total_Exports  Trade_Balance  \
Year                                                                          
2015  1.014278e+09  281406000.0  1.173375e+09   1.295684e+

In [31]:
# ============================================================================
# CELL 5: LOAD PRODUCT-LEVEL DATA (CSV VERSION)
# ============================================================================

from io import StringIO

print("\n📦 LOADING PRODUCT-LEVEL DATA")
print("="*80)

# Initialize
df_exports = pd.DataFrame()
df_imports = pd.DataFrame()
df_reexports = pd.DataFrame()
product_year_cols = []

# Load Export data
if export_file:
    print(f"\n📊 Loading EXPORTS: {export_file}")
    try:
        df_exports = pd.read_csv(StringIO(uploaded[export_file].decode('utf-8')))
        print(f"  ✅ Loaded: {len(df_exports)} rows × {len(df_exports.columns)} columns")
        print(f"  Columns: {list(df_exports.columns[:5])}...")

        if len(df_exports) > 0:
            print(f"  Sample (first row):")
            print(f"    {df_exports.iloc[0, :3].to_dict()}")
    except Exception as e:
        print(f"  ❌ Error: {str(e)}")
        df_exports = pd.DataFrame()
else:
    print(f"\n⚠️ Export file not found")

# Load Import data
if import_file:
    print(f"\n📊 Loading IMPORTS: {import_file}")
    try:
        df_imports = pd.read_csv(StringIO(uploaded[import_file].decode('utf-8')))
        print(f"  ✅ Loaded: {len(df_imports)} rows × {len(df_imports.columns)} columns")
        print(f"  Columns: {list(df_imports.columns[:5])}...")
    except Exception as e:
        print(f"  ❌ Error: {str(e)}")
        df_imports = pd.DataFrame()
else:
    print(f"\n⚠️ Import file not found")

# Load Re-export data
if reexport_file:
    print(f"\n📊 Loading RE-EXPORTS: {reexport_file}")
    try:
        df_reexports = pd.read_csv(StringIO(uploaded[reexport_file].decode('utf-8')))
        print(f"  ✅ Loaded: {len(df_reexports)} rows × {len(df_reexports.columns)} columns")
        print(f"  Columns: {list(df_reexports.columns[:5])}...")
    except Exception as e:
        print(f"  ❌ Error: {str(e)}")
        df_reexports = pd.DataFrame()
else:
    print(f"\n⚠️ Re-export file not found")

# Detect year columns
print(f"\n🔍 DETECTING YEAR COLUMNS")

if not df_exports.empty:
    print(f"  Analyzing export file columns...")
    print(f"  All columns: {list(df_exports.columns)}")

    for col in df_exports.columns:
        col_str = str(col).strip()

        # Pattern 1: Pure 4-digit year (2015, 2016, etc.)
        if col_str.isdigit() and len(col_str) == 4:
            try:
                year = int(col_str)
                if 2010 <= year <= 2030:
                    product_year_cols.append(col)
                    print(f"    ✓ Year: {col}")
            except:
                pass

        # Pattern 2: Year with text (Year_2015, 2015_Value, etc.)
        elif any(char.isdigit() for char in col_str):
            import re
            years = re.findall(r'20\d{2}', col_str)
            for year_str in years:
                try:
                    year = int(year_str)
                    if 2010 <= year <= 2030 and col not in product_year_cols:
                        product_year_cols.append(col)
                        print(f"    ✓ Year: {col}")
                except:
                    pass

if len(product_year_cols) > 0:
    print(f"\n  ✅ Found {len(product_year_cols)} year columns")
    print(f"  Range: {product_year_cols[0]} to {product_year_cols[-1]}")
else:
    print(f"\n  ⚠️ No year columns detected")
    print(f"  Make sure your CSV has columns like: 2015, 2016, 2017, etc.")

# Set flag
has_product_data = (
    (not df_exports.empty or not df_imports.empty or not df_reexports.empty)
    and len(product_year_cols) >= 2
)

print(f"\n{'='*80}")
print("PRODUCT DATA SUMMARY:")
print(f"{'='*80}")
print(f"  Exports:    {'✅' if not df_exports.empty else '❌'} ({len(df_exports)} products)")
print(f"  Imports:    {'✅' if not df_imports.empty else '❌'} ({len(df_imports)} products)")
print(f"  Re-exports: {'✅' if not df_reexports.empty else '❌'} ({len(df_reexports)} products)")
print(f"  Years:      {'✅' if len(product_year_cols) >= 2 else '❌'} ({len(product_year_cols)} columns)")
print(f"  Status:     {'✅ READY FOR ANALYSIS' if has_product_data else '❌ INCOMPLETE'}")
print(f"{'='*80}")


📦 LOADING PRODUCT-LEVEL DATA

📊 Loading EXPORTS: Oman_Export_to_KSA_HS6_10Yrs_Value.csv
  ✅ Loaded: 21 rows × 12 columns
  Columns: ['Product code', 'Product label', '2015', '2016', '2017']...
  Sample (first row):
    {'Product code': "'TOTAL", 'Product label': 'All products', '2015': ' 1,014,278,000 '}

📊 Loading IMPORTS: Oman_Import_from_KSA_HS6_10Yrs_Value.csv
  ✅ Loaded: 21 rows × 12 columns
  Columns: ['Product code', 'Product label', '2015', '2016', '2017']...

📊 Loading RE-EXPORTS: Oman_Re-export_to_KSA_HS6_10Yrs_Value.csv
  ✅ Loaded: 21 rows × 14 columns
  Columns: ['Product code', 'Product label', '2014', '2015', '2016']...

🔍 DETECTING YEAR COLUMNS
  Analyzing export file columns...
  All columns: ['Product code', 'Product label', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
    ✓ Year: 2015
    ✓ Year: 2016
    ✓ Year: 2017
    ✓ Year: 2018
    ✓ Year: 2019
    ✓ Year: 2020
    ✓ Year: 2021
    ✓ Year: 2022
    ✓ Year: 2023
    ✓ Year: 20

In [32]:
# ============================================================================
# DIAGNOSTIC: Show what files are uploaded
# ============================================================================

print("="*80)
print("DIAGNOSTIC: UPLOADED FILES")
print("="*80)

if 'uploaded' in locals() or 'uploaded' in globals():
    print(f"\n✅ Found {len(uploaded)} uploaded file(s):")

    for filename, content in uploaded.items():
        print(f"\n📄 FILE: {filename}")
        print(f"   Size: {len(content):,} bytes")
        print(f"   Type: {filename.split('.')[-1].upper()}")

        # Try to load and show structure
        if filename.endswith('.csv'):
            try:
                from io import StringIO
                df_test = pd.read_csv(StringIO(content.decode('utf-8')))
                print(f"   Rows: {len(df_test)}")
                print(f"   Columns: {len(df_test.columns)}")
                print(f"   Column names: {list(df_test.columns)}")
                print(f"\n   First 3 rows:")
                print(df_test.head(3))
            except Exception as e:
                print(f"   ❌ Error reading: {str(e)[:100]}")

        print("-" * 80)
else:
    print("\n❌ No 'uploaded' variable found!")
    print("   Please run Cell 2 to upload your files first")

print("="*80)

DIAGNOSTIC: UPLOADED FILES

✅ Found 4 uploaded file(s):

📄 FILE: Oman_Export_to_KSA_HS6_10Yrs_Value.csv
   Size: 3,556 bytes
   Type: CSV
   Rows: 21
   Columns: 12
   Column names: ['Product code', 'Product label', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']

   First 3 rows:
  Product code                                      Product label  \
0       'TOTAL                                       All products   
1      '340250  Surface-active preparations, washing preparati...   
2      '711319  Articles of jewellery and parts thereof, of pr...   

              2015           2016             2017             2018  \
0   1,014,278,000    818,044,000    1,314,021,000    1,661,444,000    
1               -              -                -                -    
2               -              -           33,000        5,805,000    

              2019             2020             2021             2022  \
0   1,603,346,000    1,488,146,000    1,728,639,000

In [47]:
# ============================================================================
# CELL 6: BCG PORTFOLIO ANALYSIS WITH STRATEGIC NARRATIVES
# ============================================================================

import time
start_time = time.time()

print("\n" + "="*80)
print("⭐ BCG PORTFOLIO ANALYSIS: UNDERSTANDING YOUR EXPORT PORTFOLIO")
print("="*80)

print("""
📖 WHAT IS BCG MATRIX?
The Boston Consulting Group (BCG) Matrix is a strategic tool that helps us
understand which products deserve your attention and resources. Think of it
as a map showing where each product sits in terms of two critical dimensions:

1. GROWTH RATE: How fast is demand for this product growing?
2. MARKET SHARE: How much of the export pie does this product represent?

These two dimensions create four strategic categories:

🌟 STARS: High growth + High market share
   → Your future. These are winners you should bet on heavily.

💰 CASH COWS: Low growth + High market share
   → Your present. Mature products generating stable revenue.

❓ QUESTION MARKS: High growth + Low market share
   → Your opportunities. Fast-growing but small - invest selectively.

🐕 DOGS: Low growth + Low market share
   → Your burden. Consider divesting to free up resources.
""")

def calculate_product_metrics(df_products, year_cols, max_products=1000):
    """Calculate comprehensive metrics for BCG analysis"""

    if df_products.empty or len(year_cols) < 2:
        return pd.DataFrame()

    num_to_process = min(max_products, len(df_products))
    print(f"  📊 Analyzing {num_to_process} products across {len(year_cols)} years...")

    results = []

    # Identify product column
    product_col = None
    for col in df_products.columns[:3]:
        col_lower = str(col).lower()
        if any(kw in col_lower for kw in ['label', 'product', 'description', 'commodity']):
            product_col = col
            break

    if product_col is None:
        product_col = df_products.columns[1] if len(df_products.columns) > 1 else df_products.columns[0]

    print(f"  ✓ Using '{product_col}' as product identifier")

    # Process each product
    for idx in range(num_to_process):
        if idx % 200 == 0 and idx > 0:
            print(f"    → Processed {idx}/{num_to_process} products...")

        try:
            row = df_products.iloc[idx]
            product_name = str(row[product_col])[:150]

            # Skip empty/total rows
            if pd.isna(product_name) or product_name.strip() == '' or product_name.lower() == 'nan':
                continue
            if 'total' in product_name.lower() or 'all products' in product_name.lower():
                continue

            hs_code = str(row.iloc[0]) if len(df_products.columns) > 1 else ''

            # Get year values
            first_year = year_cols[0]
            last_year = year_cols[-1]

            start_val = pd.to_numeric(row.get(first_year, 0), errors='coerce')
            end_val = pd.to_numeric(row.get(last_year, 0), errors='coerce')

            # Calculate totals
            year_values = []
            for y in year_cols:
                val = pd.to_numeric(row.get(y, 0), errors='coerce')
                if pd.notna(val) and val > 0:
                    year_values.append(val)

            if len(year_values) == 0:
                continue

            total_val = sum(year_values)
            avg_val = np.mean(year_values)

            if total_val < 1000:
                continue

            # Calculate CAGR
            cagr = 0
            if pd.notna(start_val) and pd.notna(end_val) and start_val > 0 and end_val > 0:
                periods = len(year_cols) - 1
                cagr = (((end_val / start_val) ** (1 / periods)) - 1) * 100

                if abs(cagr) > 300:
                    continue
            elif start_val == 0 and end_val > 0:
                cagr = 100

            results.append({
                'HS_Code': hs_code,
                'Product': product_name,
                'Start_Value': start_val if pd.notna(start_val) else 0,
                'End_Value': end_val if pd.notna(end_val) else 0,
                'Total_Value': total_val,
                'Avg_Annual_Value': avg_val,
                'CAGR': cagr
            })

        except Exception as e:
            continue

    df_result = pd.DataFrame(results)

    if not df_result.empty:
        df_result = df_result.sort_values('Total_Value', ascending=False).reset_index(drop=True)
        print(f"  ✅ Successfully analyzed {len(df_result)} products")

    return df_result

# Initialize
bcg_export = pd.DataFrame()
bcg_import = pd.DataFrame()
bcg_summary = pd.DataFrame()

# ============================================================================
# PROCESS EXPORTS
# ============================================================================

if has_product_data and not df_exports.empty and len(product_year_cols) >= 2:

    print(f"\n{'='*80}")
    print("📦 ANALYZING YOUR EXPORT PORTFOLIO")
    print(f"{'='*80}")
    print(f"   Dataset: {len(df_exports)} products from {product_year_cols[0]} to {product_year_cols[-1]}")

    bcg_export = calculate_product_metrics(df_exports, product_year_cols, max_products=1000)

    if not bcg_export.empty and len(bcg_export) >= 4:

        print(f"\n🎯 APPLYING BCG CLASSIFICATION FRAMEWORK")
        print(f"{'='*80}")

        # Calculate market share
        total_value = bcg_export['Total_Value'].sum()
        bcg_export['Market_Share'] = (bcg_export['Total_Value'] / total_value) * 100

        # Classification thresholds
        median_growth = bcg_export['CAGR'].median()
        median_share = bcg_export['Market_Share'].median()

        print(f"""
📏 CLASSIFICATION THRESHOLDS:
   • Growth threshold: {median_growth:.2f}% CAGR (median growth rate)
   • Market share threshold: {median_share:.3f}% (median share)

   Products above both thresholds = Stars ⭐
   Products above share only = Cash Cows 💰
   Products above growth only = Question Marks ❓
   Products below both = Dogs 🐕
""")

        # BCG Classification
        def classify_bcg(row):
            growth = row['CAGR']
            share = row['Market_Share']

            if growth >= median_growth and share >= median_share:
                return 'Star'
            elif growth < median_growth and share >= median_share:
                return 'Cash Cow'
            elif growth >= median_growth and share < median_share:
                return 'Question Mark'
            else:
                return 'Dog'

        bcg_export['BCG_Category'] = bcg_export.apply(classify_bcg, axis=1)

        # Strategic narratives for each category
        def get_strategic_narrative(category, cagr):
            narratives = {
                'Star': {
                    'priority': 'CRITICAL',
                    'action': 'Invest & Grow',
                    'explanation': 'These products are your future revenue engines. They\'re growing fast AND already have strong market position.',
                    'strategy': 'Pour resources here: marketing, capacity, distribution. Protect and expand market share.',
                    'risk': 'If you don\'t invest, competitors will capture this growth.'
                },
                'Cash Cow': {
                    'priority': 'MEDIUM',
                    'action': 'Maintain & Harvest',
                    'explanation': 'Mature products with strong position but slow growth. Your current profit generators.',
                    'strategy': 'Maintain quality and market position. Use profits to fund Stars and promising Question Marks.',
                    'risk': 'Don\'t over-invest. These products are mature and won\'t grow significantly.'
                },
                'Question Mark': {
                    'priority': 'HIGH' if cagr > 20 else 'LOW',
                    'action': 'Evaluate Selectively' if cagr > 20 else 'Monitor or Exit',
                    'explanation': 'Growing markets but you have small share. The "dilemma" products.',
                    'strategy': 'For high-growth ones: Invest heavily to capture share or exit. For low-growth: Minimize investment.',
                    'risk': 'These consume cash. If you can\'t gain share, they become Dogs.'
                },
                'Dog': {
                    'priority': 'VERY LOW',
                    'action': 'Consider Divestment',
                    'explanation': 'Low growth AND low share. They tie up resources with little return.',
                    'strategy': 'Divest or minimize resources. Reallocate freed resources to Stars.',
                    'risk': 'Keeping Dogs drains resources from better opportunities.'
                }
            }
            return narratives.get(category, {})

        bcg_export['Strategic_Narrative'] = bcg_export.apply(
            lambda row: get_strategic_narrative(row['BCG_Category'], row['CAGR']),
            axis=1
        )

        # Generate summary
        bcg_summary = bcg_export.groupby('BCG_Category').agg({
            'Product': 'count',
            'Total_Value': 'sum',
            'Avg_Annual_Value': 'mean',
            'CAGR': 'mean',
            'Market_Share': 'sum'
        }).round(2)

        bcg_summary.columns = ['Count', 'Total_Value_$', 'Avg_Annual_$', 'Avg_CAGR_%', 'Total_Share_%']

        # Sort strategically
        category_order = ['Star', 'Question Mark', 'Cash Cow', 'Dog']
        bcg_summary = bcg_summary.reindex([cat for cat in category_order if cat in bcg_summary.index])

        print(f"\n{'='*80}")
        print("📊 YOUR PORTFOLIO AT A GLANCE")
        print(f"{'='*80}")
        print(f"{'Category':<15} {'Count':>6} {'Total Value':>15} {'Avg CAGR':>12} {'Portfolio %':>12}")
        print(f"{'-'*80}")

        for cat, row in bcg_summary.iterrows():
            print(f"{cat:<15} {row['Count']:>6.0f} ${row['Total_Value_$']/1e9:>13.2f}B {row['Avg_CAGR_%']:>10.1f}% {row['Total_Share_%']:>11.1f}%")

        print(f"{'='*80}")

        # Portfolio health assessment
        print(f"\n🏥 PORTFOLIO HEALTH CHECK:")

        star_value = bcg_summary.loc['Star', 'Total_Value_$'] if 'Star' in bcg_summary.index else 0
        total_portfolio_value = bcg_summary['Total_Value_$'].sum()
        star_percentage = (star_value / total_portfolio_value * 100) if total_portfolio_value > 0 else 0

        dog_value = bcg_summary.loc['Dog', 'Total_Value_$'] if 'Dog' in bcg_summary.index else 0
        dog_percentage = (dog_value / total_portfolio_value * 100) if total_portfolio_value > 0 else 0

        print(f"\n   Stars represent {star_percentage:.1f}% of your portfolio value")
        if star_percentage > 30:
            print(f"   ✅ HEALTHY: Strong future revenue pipeline")
        elif star_percentage > 15:
            print(f"   ⚠️ MODERATE: Consider increasing investment in high-growth products")
        else:
            print(f"   ❌ CONCERNING: Weak future growth prospects - urgent action needed")

        print(f"\n   Dogs represent {dog_percentage:.1f}% of your portfolio value")
        if dog_percentage < 10:
            print(f"   ✅ HEALTHY: Minimal drag from underperforming products")
        elif dog_percentage < 20:
            print(f"   ⚠️ MODERATE: Consider divestment to free resources")
        else:
            print(f"   ❌ CONCERNING: Too many resources tied up in weak products")

        # ====================================================================
        # STARS - The Narrative
        # ====================================================================
        stars = bcg_export[bcg_export['BCG_Category'] == 'Star'].nlargest(15, 'Total_Value')

        if not stars.empty:
            print(f"\n{'='*80}")
            print(f"🌟 YOUR STAR PRODUCTS: THE FUTURE OF YOUR EXPORT BUSINESS")
            print(f"{'='*80}")

            print(f"""
📖 WHY STARS MATTER:
Stars are products in high-growth markets where you already have strong position.
These are your "sweet spot" - the products that will drive revenue growth for years.

Think of Stars as:
- Your competitive advantage in action
- Markets where you're winning AND the market is expanding
- The products that justify your strategic investments

THE NUMBERS:
- Total Stars: {len(bcg_export[bcg_export['BCG_Category'] == 'Star'])} products
- Combined Value: ${stars['Total_Value'].sum()/1e9:.2f}B ({stars['Market_Share'].sum():.1f}% of exports)
- Average Growth: {stars['CAGR'].mean():.1f}% CAGR

💡 STRATEGIC IMPERATIVE:
Invest aggressively in Stars. They're expensive (high growth requires investment),
but they're building your market leadership in tomorrow's biggest opportunities.
Starve Stars of resources and competitors will capture the growth.
""")

            print(f"\n{'─'*80}")
            print(f"TOP 15 STAR PRODUCTS - YOUR INVESTMENT PRIORITIES:")
            print(f"{'─'*80}")

            for i, (idx, row) in enumerate(stars.iterrows(), 1):
                narrative = row['Strategic_Narrative']

                print(f"\n{'█'*80}")
                print(f"#{i} STAR PRODUCT")
                print(f"{'█'*80}")
                print(f"📦 Product: {row['Product']}")
                print(f"   HS Code: {row['HS_Code']}")

                print(f"\n💰 FINANCIAL PROFILE:")
                print(f"   • Total Value (cumulative): ${row['Total_Value']/1e6:,.1f}M")
                print(f"   • Average Annual Value: ${row['Avg_Annual_Value']/1e6:.1f}M/year")
                print(f"   • Market Share: {row['Market_Share']:.2f}% of total exports")

                print(f"\n📈 GROWTH PROFILE:")
                print(f"   • CAGR: {row['CAGR']:+.1f}% (Growing from ${row['Start_Value']/1e6:.1f}M to ${row['End_Value']/1e6:.1f}M)")

                if row['CAGR'] > 20:
                    growth_desc = "🚀 EXCEPTIONAL GROWTH - Top priority investment"
                elif row['CAGR'] > 10:
                    growth_desc = "📈 STRONG GROWTH - Maintain momentum"
                else:
                    growth_desc = "📊 SOLID GROWTH - Monitor competitors"
                print(f"   • Assessment: {growth_desc}")

                print(f"\n🎯 STRATEGIC GUIDANCE:")
                print(f"   Priority: {narrative['priority']}")
                print(f"   Action: {narrative['action']}")
                print(f"   \n   💡 Why this matters:")
                print(f"   {narrative['explanation']}")
                print(f"   \n   📋 What to do:")
                print(f"   {narrative['strategy']}")
                print(f"   \n   ⚠️ Risk if ignored:")
                print(f"   {narrative['risk']}")

        # ====================================================================
        # QUESTION MARKS - The Narrative
        # ====================================================================
        question_marks = bcg_export[bcg_export['BCG_Category'] == 'Question Mark']
        high_growth_qm = question_marks[question_marks['CAGR'] > 15].nlargest(10, 'CAGR')

        if not high_growth_qm.empty:
            print(f"\n{'='*80}")
            print(f"❓ QUESTION MARKS: YOUR STRATEGIC DILEMMAS")
            print(f"{'='*80}")

            print(f"""
📖 UNDERSTANDING QUESTION MARKS:
These products are in fast-growing markets, but you have low market share.
They're called "Question Marks" because they force difficult decisions:

THE DILEMMA:
- Option 1: Invest heavily to capture share → potential Star
- Option 2: Exit and reallocate resources → avoid cash drain

Question Marks are "cash traps" - they need investment to grow, but there's
no guarantee you'll gain share. Some will become Stars. Most will become Dogs.

THE NUMBERS:
- High-growth Question Marks (>15% CAGR): {len(high_growth_qm)} products
- Combined Value: ${high_growth_qm['Total_Value'].sum()/1e9:.2f}B
- Average Growth: {high_growth_qm['CAGR'].mean():.1f}% CAGR

💡 STRATEGIC CHOICE:
For each Question Mark, ask: "Can we realistically gain share?"
If YES → Invest aggressively (potential Star)
If NO → Exit quickly (avoid becoming a Dog)
If UNSURE → Small pilot investment to test market response
""")

            print(f"\n{'─'*80}")
            print(f"TOP 10 HIGH-GROWTH QUESTION MARKS - INVESTMENT DECISIONS NEEDED:")
            print(f"{'─'*80}")

            for i, (idx, row) in enumerate(high_growth_qm.iterrows(), 1):
                narrative = row['Strategic_Narrative']

                print(f"\n{'▓'*80}")
                print(f"#{i} QUESTION MARK")
                print(f"{'▓'*80}")
                print(f"📦 Product: {row['Product']}")

                print(f"\n🎲 THE OPPORTUNITY:")
                print(f"   • Market Growth: {row['CAGR']:+.1f}% CAGR ⚡ (FAST-GROWING MARKET)")
                print(f"   • Current Value: ${row['Total_Value']/1e6:,.1f}M")
                print(f"   • Your Market Share: {row['Market_Share']:.2f}% (LOW - Room to grow)")

                print(f"\n🤔 THE STRATEGIC QUESTION:")
                print(f"   This market is growing rapidly, but you're a small player.")
                print(f"   Can you invest enough to capture meaningful share?")

                potential_value = row['Avg_Annual_Value'] * (1 + row['CAGR']/100) ** 3
                print(f"\n   💰 If you capture share, potential 3-year value: ${potential_value/1e6:.1f}M/year")
                print(f"   ⚠️ If you don't, competitors capture this ${potential_value/1e6:.1f}M opportunity")

                print(f"\n📋 DECISION FRAMEWORK:")
                print(f"   ✅ INVEST IF: You have competitive advantage, resources, and management focus")
                print(f"   ❌ EXIT IF: Competitors are entrenched, barriers are high, or you lack resources")
                print(f"   🔬 TEST IF: Uncertain - run small pilot to test market response")

        # ====================================================================
        # CASH COWS - The Narrative
        # ====================================================================
        cash_cows = bcg_export[bcg_export['BCG_Category'] == 'Cash Cow'].nlargest(10, 'Total_Value')

        if not cash_cows.empty:
            print(f"\n{'='*80}")
            print(f"💰 CASH COWS: YOUR PROFIT GENERATORS")
            print(f"{'='*80}")

            print(f"""
📖 UNDERSTANDING CASH COWS:
These are mature products where you have strong market position but growth is slow.
Think of them as your "steady income" - reliable, profitable, low-risk.

WHY CASH COWS MATTER:
- They generate stable profits without much investment
- They fund your investments in Stars and Question Marks
- They provide financial stability during downturns

THE TRAP TO AVOID:
Don't over-invest in Cash Cows. They're mature - more investment won't
create much growth. Use their profits to fund Stars instead.

THE NUMBERS:
- Total Cash Cows: {len(bcg_export[bcg_export['BCG_Category'] == 'Cash Cow'])} products
- Combined Value: ${cash_cows['Total_Value'].sum()/1e9:.2f}B
- Average Growth: {cash_cows['CAGR'].mean():.1f}% CAGR (Mature/stable)

💡 STRATEGIC APPROACH:
Maintain quality and market position. Harvest profits. Don't chase growth.
Use Cash Cow profits to fund Stars and promising Question Marks.
""")

            print(f"\n{'─'*80}")
            print(f"TOP 10 CASH COWS:")
            print(f"{'─'*80}")

            for i, (idx, row) in enumerate(cash_cows.iterrows(), 1):
                print(f"\n{i:2}. {row['Product'][:70]}")
                print(f"    💰 Value: ${row['Total_Value']/1e6:,.0f}M | 📊 Share: {row['Market_Share']:.2f}% | 📈 Growth: {row['CAGR']:+.1f}%")
                print(f"    Strategy: Maintain position, harvest profits, fund Stars with cash flow")

        # ====================================================================
        # DOGS - The Narrative
        # ====================================================================
        dogs = bcg_export[bcg_export['BCG_Category'] == 'Dog']

        if not dogs.empty:
            print(f"\n{'='*80}")
            print(f"🐕 DOG PRODUCTS: THE RESOURCE DRAIN")
            print(f"{'='*80}")

            print(f"""
📖 UNDERSTANDING DOGS:
Low growth markets where you have low share. These are usually:
- Commodity products with intense competition
- Declining industries
- Markets where you never gained traction

THE PROBLEM WITH DOGS:
They consume management time, working capital, and resources that could
be invested in Stars. Every dollar in Dogs is a dollar NOT in Stars.

THE NUMBERS:
- Total Dogs: {len(dogs)} products
- Combined Value: ${dogs['Total_Value'].sum()/1e9:.2f}B ({dog_percentage:.1f}% of portfolio)
- Average Growth: {dogs['CAGR'].mean():.1f}% CAGR

💡 STRATEGIC IMPERATIVE:
Divest or minimize resources. Some exceptions:
- Strategic importance (e.g., completes product line)
- Social/political considerations
- Low marginal cost to maintain

Otherwise, exit and reallocate resources to Stars.
""")

            top_dogs = dogs.nlargest(10, 'Total_Value')
            if not top_dogs.empty:
                print(f"\n Top 10 Dogs by value (evaluate case-by-case):")
                for i, (idx, row) in enumerate(top_dogs.iterrows(), 1):
                    print(f"  {i:2}. {row['Product'][:65]} - ${row['Total_Value']/1e6:.0f}M")
                    print(f"      → Question: Why are we still here? Strategic value or exit?")

        print(f"\n{'='*80}")
        print(f"✅ BCG ANALYSIS COMPLETE")
        print(f"{'='*80}")

    else:
        print(f"  ⚠️ Need at least 4 products for BCG classification (found {len(bcg_export)})")

# Process imports (simplified narrative)
if has_product_data and not df_imports.empty and len(product_year_cols) >= 2:
    print(f"\n{'='*80}")
    print("📥 IMPORT ANALYSIS (For Context)")
    print(f"{'='*80}")

    bcg_import = calculate_product_metrics(df_imports, product_year_cols, max_products=1000)

    if not bcg_import.empty:
        total_import_value = bcg_import['Total_Value'].sum()
        bcg_import['Market_Share'] = (bcg_import['Total_Value'] / total_import_value) * 100

        print(f"  ✅ {len(bcg_import)} import products analyzed")
        print(f"\n  Understanding your imports helps identify:")
        print(f"  • Supply chain dependencies")
        print(f"  • Potential import substitution opportunities")
        print(f"  • Products to localize production")

elapsed = time.time() - start_time

print(f"\n{'='*80}")
print(f"⏱️ Analysis completed in {elapsed:.1f} seconds")
print(f"{'='*80}")


⭐ BCG PORTFOLIO ANALYSIS: UNDERSTANDING YOUR EXPORT PORTFOLIO

📖 WHAT IS BCG MATRIX?
The Boston Consulting Group (BCG) Matrix is a strategic tool that helps us 
understand which products deserve your attention and resources. Think of it 
as a map showing where each product sits in terms of two critical dimensions:

1. GROWTH RATE: How fast is demand for this product growing?
2. MARKET SHARE: How much of the export pie does this product represent?

These two dimensions create four strategic categories:

🌟 STARS: High growth + High market share
   → Your future. These are winners you should bet on heavily.
   
💰 CASH COWS: Low growth + High market share  
   → Your present. Mature products generating stable revenue.
   
❓ QUESTION MARKS: High growth + Low market share
   → Your opportunities. Fast-growing but small - invest selectively.
   
🐕 DOGS: Low growth + Low market share
   → Your burden. Consider divesting to free up resources.


📦 ANALYZING YOUR EXPORT PORTFOLIO
   Dataset: 21 

In [48]:
# ============================================================================
# CELL 7: HHI CONCENTRATION ANALYSIS WITH STRATEGIC NARRATIVES
# ============================================================================

print("\n" + "="*80)
print("📊 HHI CONCENTRATION ANALYSIS: UNDERSTANDING YOUR RISK PROFILE")
print("="*80)

print("""
📖 WHAT IS HHI (HERFINDAHL-HIRSCHMAN INDEX)?

Imagine your export portfolio as a basket of eggs. HHI tells you how many
baskets you have - and what happens if you drop one.

HHI is a concentration index that answers: "How dependent are we on a few products?"

THE MATH (simplified):
HHI = Sum of squared market shares (as percentages)
Example: If 3 products have 50%, 30%, and 20% share:
HHI = 50² + 30² + 20² = 2,500 + 900 + 400 = 3,800

WHAT THE NUMBERS MEAN:
┌─────────────────────────────────────────────────────────────┐
│ HHI < 1,500  │ COMPETITIVE (Low Risk)                      │
│              │ Many products, well diversified             │
│              │ Like having 20+ eggs in 20 different baskets│
├─────────────────────────────────────────────────────────────┤
│ HHI 1,500-   │ MODERATE CONCENTRATION (Medium Risk)        │
│     2,500    │ Some products dominate                      │
│              │ Like having 10 eggs in 5-6 baskets          │
├─────────────────────────────────────────────────────────────┤
│ HHI > 2,500  │ HIGH CONCENTRATION (High Risk)              │
│              │ Few products dominate                       │
│              │ Like having all eggs in 2-3 baskets         │
└─────────────────────────────────────────────────────────────┘

WHY IT MATTERS:
High HHI = High risk. If one major product fails (demand drops, competitor
emerges, technology changes), your entire export revenue is at risk.

Real-world examples:
- Venezuela (oil): HHI ~9,000 - collapsed when oil prices fell
- Switzerland (diversified): HHI ~800 - stable through crises
""")

def calculate_hhi(df_products, value_col='Total_Value'):
    """Calculate HHI with detailed breakdown"""
    if df_products.empty or value_col not in df_products.columns:
        return 0, "N/A", {}

    total = df_products[value_col].sum()
    if total == 0:
        return 0, "N/A", {}

    # Calculate shares
    shares = (df_products[value_col] / total) * 100
    hhi = (shares ** 2).sum()

    # Interpretation
    if hhi < 1500:
        interpretation = "Low concentration (Competitive/Diversified)"
        risk_level = "LOW RISK"
        color = "🟢"
    elif hhi < 2500:
        interpretation = "Moderate concentration"
        risk_level = "MEDIUM RISK"
        color = "🟡"
    else:
        interpretation = "High concentration (Oligopolistic)"
        risk_level = "HIGH RISK"
        color = "🔴"

    # Additional metrics
    metrics = {
        'risk_level': risk_level,
        'color': color,
        'hhi': hhi,
        'interpretation': interpretation
    }

    return hhi, interpretation, metrics

# ============================================================================
# EXPORT CONCENTRATION
# ============================================================================

if not bcg_export.empty:
    print(f"\n{'='*80}")
    print("📤 YOUR EXPORT CONCENTRATION PROFILE")
    print(f"{'='*80}")

    hhi_exports, concentration, metrics = calculate_hhi(bcg_export, 'Total_Value')

    print(f"\n{metrics['color']} HHI INDEX: {hhi_exports:,.0f}")
    print(f"   Classification: {concentration}")
    print(f"   Risk Level: {metrics['risk_level']}")

    # Detailed interpretation
    print(f"\n📖 WHAT THIS MEANS FOR YOUR BUSINESS:")

    if hhi_exports < 1500:
        print(f"""
✅ WELL-DIVERSIFIED PORTFOLIO (HHI: {hhi_exports:.0f})

THE GOOD NEWS:
- Your exports are spread across many products
- No single product dominates your revenue
- You're protected against product-specific shocks
- Similar to a mutual fund - diversification reduces risk

REAL-WORLD COMPARISON:
This is like Germany or Japan - diversified manufacturing economies that
weather individual product cycles well.

STRATEGIC IMPLICATION:
You have flexibility. You can:
- Take risks on new products without jeopardizing stability
- Weather competitive threats in individual products
- Ride out demand fluctuations more easily

⚠️ ONE CAUTION:
Don't mistake diversification for strength. Make sure you're diversified
across STRONG products, not just many weak ones. Check your BCG Stars!
""")

    elif hhi_exports < 2500:
        print(f"""
⚠️ MODERATELY CONCENTRATED (HHI: {hhi_exports:.0f})

THE SITUATION:
- A few products account for most of your exports
- You have some diversification, but vulnerabilities exist
- Like having a portfolio tilted toward a few sectors

THE RISK:
If demand drops or competition intensifies in your top products, you'll
feel significant revenue impact. Not catastrophic, but material.

REAL-WORLD COMPARISON:
Similar to South Korea in the 1990s - concentrated in electronics and
autos, but had backup sectors.

STRATEGIC RECOMMENDATIONS:
1. Monitor your top 5-10 products closely (they drive your results)
2. Invest in diversifying into related products (de-risk gradually)
3. For top products: invest in quality, innovation, and market defense
4. For emerging products: accelerate growth to increase diversification

TARGET:
Work toward HHI < 1,500 over 3-5 years through strategic diversification.
""")

    else:  # HHI > 2500
        print(f"""
🔴 HIGHLY CONCENTRATED - URGENT ACTION NEEDED (HHI: {hhi_exports:.0f})

THE PROBLEM:
- A very small number of products dominate your exports
- You're extremely vulnerable to product-specific risks
- Like having most wealth in 1-2 stocks

THE RISKS:
- Technology disruption in key product → revenue collapse
- New competitor in key product → market share loss
- Demand shift in key product → export crisis
- Trade policy change → concentrated impact

REAL-WORLD WARNINGS:
- Venezuela (oil-dependent): Economic crisis when oil prices fell
- Nokia (mobile phones): Collapsed when smartphones emerged
- One-product economies: Vulnerable to every market shift

This is NOT sustainable. You MUST diversify.

🚨 URGENT STRATEGIC PRIORITIES:

1. SHORT-TERM (0-12 months):
   • Protect current key products (competitors smell weakness)
   • Fast-track 3-5 diversification projects
   • Create crisis response plan for top product scenarios
   • Establish export diversification task force

2. MEDIUM-TERM (1-3 years):
   • Target: Reduce HHI below 2,000
   • Launch 10+ new product export initiatives
   • Enter adjacent markets with existing products
   • Invest in R&D for product portfolio expansion

3. LONG-TERM (3-5 years):
   • Target: Achieve HHI below 1,500
   • Build diversified export base across sectors
   • Create institutional capability for market development

FINANCIAL ANALOGY:
You're like someone with 80% of wealth in one stock. Financial advisors
would say this is reckless. Same applies to your export portfolio.
""")

    # Concentration ratios
    if len(bcg_export) > 0:
        top3_share = bcg_export.nlargest(3, 'Market_Share')['Market_Share'].sum()
        top5_share = bcg_export.nlargest(5, 'Market_Share')['Market_Share'].sum()
        top10_share = bcg_export.nlargest(10, 'Market_Share')['Market_Share'].sum()

        print(f"\n📊 CONCENTRATION RATIOS (How much do top products control?):")
        print(f"{'─'*80}")
        print(f"   Top 3 products:  {top3_share:>5.1f}% of exports", end="")
        if top3_share > 50:
            print(f"  🔴 DANGER: Over half your exports!")
        elif top3_share > 30:
            print(f"  🟡 WARNING: High dependency")
        else:
            print(f"  🟢 HEALTHY: Well-distributed")

        print(f"   Top 5 products:  {top5_share:>5.1f}% of exports", end="")
        if top5_share > 70:
            print(f"  🔴 DANGER: Extreme concentration")
        elif top5_share > 50:
            print(f"  🟡 WARNING: Moderate concentration")
        else:
            print(f"  🟢 HEALTHY: Good distribution")

        print(f"   Top 10 products: {top10_share:>5.1f}% of exports", end="")
        if top10_share > 85:
            print(f"  🔴 DANGER: Almost everything!")
        elif top10_share > 70:
            print(f"  🟡 WARNING: Heavy concentration")
        else:
            print(f"  🟢 HEALTHY: Balanced portfolio")

        print(f"\n💡 RULE OF THUMB:")
        print(f"   Healthy portfolio: Top 10 products < 60% of exports")
        print(f"   Your portfolio: Top 10 products = {top10_share:.1f}%")

        if top10_share > 60:
            print(f"   → Action needed: Diversification is priority")
        else:
            print(f"   → Well-diversified: Maintain balance")

    # Top products
    print(f"\n{'='*80}")
    print(f"🏆 YOUR TOP 10 PRODUCTS BY MARKET SHARE")
    print(f"{'='*80}")
    print(f"These products drive your export results. Monitor them closely.")

    top_products = bcg_export.nlargest(10, 'Market_Share')

    cumulative_share = 0
    for i, (idx, row) in enumerate(top_products.iterrows(), 1):
        cumulative_share += row['Market_Share']

        print(f"\n{i:2}. {row['Product'][:70]}")
        print(f"    📊 Market Share: {row['Market_Share']:.2f}% (Cumulative: {cumulative_share:.1f}%)")
        print(f"    💰 Value: ${row['Total_Value']/1e6:,.1f}M")
        print(f"    📈 Growth: {row['CAGR']:+.1f}% CAGR")

        if 'BCG_Category' in row:
            print(f"    🎯 BCG Category: {row['BCG_Category']}", end="")
            if row['BCG_Category'] == 'Dog' and row['Market_Share'] > 5:
                print(f" ⚠️ (High share BUT low growth - danger!)")
            elif row['BCG_Category'] == 'Star':
                print(f" ✅ (Perfect - high share AND high growth)")
            else:
                print()

        # Risk assessment for top products
        if i <= 3 and row['Market_Share'] > 15:
            print(f"    ⚠️ RISK: This product alone is {row['Market_Share']:.1f}% of exports")
            print(f"       → Create contingency plan: What if demand drops 20%?")

# Import concentration
if not bcg_import.empty:
    print(f"\n{'='*80}")
    print("📥 YOUR IMPORT CONCENTRATION PROFILE")
    print(f"{'='*80}")

    hhi_imports, concentration_imp, metrics_imp = calculate_hhi(bcg_import, 'Total_Value')

    print(f"\n{metrics_imp['color']} HHI INDEX: {hhi_imports:,.0f}")
    print(f"   Classification: {concentration_imp}")
    print(f"   Risk Level: {metrics_imp['risk_level']}")

    print(f"\n📖 IMPORT CONCENTRATION IMPLICATIONS:")

    if hhi_imports > 2500:
        print(f"""
⚠️ Your imports are highly concentrated. This creates:
- Supply chain vulnerability (dependence on few products)
- Negotiating weakness (suppliers know you depend on them)
- Price risk (limited alternatives if prices spike)

OPPORTUNITY:
- High-value imports might be opportunities for local production
- "Import substitution" could reduce dependency AND create exports
""")
    else:
        print(f"""
✅ Your imports are well-diversified. This means:
- Supply chain resilience (many alternatives available)
- Negotiating power (can switch suppliers)
- Lower risk from import price spikes
""")

# Comparative analysis
if not bcg_export.empty and not bcg_import.empty:
    print(f"\n{'='*80}")
    print("⚖️ EXPORT vs IMPORT CONCENTRATION")
    print(f"{'='*80}")

    print(f"\nExport HHI: {hhi_exports:,.0f} ({concentration})")
    print(f"Import HHI: {hhi_imports:,.0f} ({concentration_imp})")

    if hhi_exports > hhi_imports:
        diff = hhi_exports - hhi_imports
        print(f"\n📊 Exports are MORE concentrated than imports (+{diff:.0f} points)")
        print(f"""
WHAT THIS MEANS:
Your export revenue is more dependent on few products than your import
spending is. This is concerning because:

- Export concentration = Revenue risk (few products drive earnings)
- Import diversification = Cost flexibility (many options to buy)

ANALOGY:
You're like a company with one main customer (risky) but many suppliers
(good). If that customer leaves, you're in trouble.

STRATEGIC PRIORITY:
Export diversification is MORE urgent than import management.
""")
    else:
        diff = hhi_imports - hhi_exports
        print(f"\n📊 Imports are MORE concentrated than exports (+{diff:.0f} points)")
        print(f"""
WHAT THIS MEANS:
Your imports are more concentrated than exports. This suggests:

- Supply chain risk (dependent on few import products)
- Possible import substitution opportunities

OPPORTUNITY:
Your concentrated imports might be:
- Raw materials you could source locally
- Inputs you could manufacture domestically
- Products to target for "backward integration"

Concentrated imports + Diversified exports = Supply chain vulnerability
""")

print(f"\n{'='*80}")
print(f"💡 FINAL STRATEGIC TAKEAWAY")
print(f"{'='*80}")

if not bcg_export.empty:
    print(f"""
Your export portfolio's HHI of {hhi_exports:.0f} tells a story about risk.

If HHI is high: You're betting everything on a few products. This worked
in stable times, but global markets are volatile. One disruption (tech
change, new competitor, policy shift) could devastate exports.

If HHI is low: You're diversified, which is good. But check your BCG
matrix - make sure you're diversified across STRONG products (Stars),
not just many weak ones (Dogs).

THE BALANCE:
- Too concentrated (HHI > 2,500) = Risky (all eggs in one basket)
- Too diversified (HHI < 500) = Unfocused (no competitive advantage)
- Sweet spot (HHI 800-1,500) = Diversified strength

REMEMBER:
HHI is a RISK metric. Low HHI doesn't guarantee success - it just means
you're not over-dependent. You still need strong products (Stars) to grow.

Use HHI with BCG Matrix:
- HHI shows WHERE risk is (product concentration)
- BCG shows WHICH products to invest in (Stars vs Dogs)
- Together, they guide portfolio strategy
""")

print(f"\n{'='*80}")
print(f"✅ HHI ANALYSIS COMPLETE")
print(f"{'='*80}")


📊 HHI CONCENTRATION ANALYSIS: UNDERSTANDING YOUR RISK PROFILE

📖 WHAT IS HHI (HERFINDAHL-HIRSCHMAN INDEX)?

Imagine your export portfolio as a basket of eggs. HHI tells you how many
baskets you have - and what happens if you drop one.

HHI is a concentration index that answers: "How dependent are we on a few products?"

THE MATH (simplified):
HHI = Sum of squared market shares (as percentages)
Example: If 3 products have 50%, 30%, and 20% share:
HHI = 50² + 30² + 20² = 2,500 + 900 + 400 = 3,800

WHAT THE NUMBERS MEAN:
┌─────────────────────────────────────────────────────────────┐
│ HHI < 1,500  │ COMPETITIVE (Low Risk)                      │
│              │ Many products, well diversified             │
│              │ Like having 20+ eggs in 20 different baskets│
├─────────────────────────────────────────────────────────────┤
│ HHI 1,500-   │ MODERATE CONCENTRATION (Medium Risk)        │
│     2,500    │ Some products dominate                      │
│              │ Like having 10 

In [49]:
# ============================================================================
# CELL 8: CALCULATE AGGREGATE METRICS
# ============================================================================

print("\n📊 CALCULATING COMPREHENSIVE METRICS")
print("="*80)

# CAGR function
def calc_cagr(start, end, periods):
    if start == 0 or end == 0:
        return 0.0
    return (((end / start) ** (1 / periods)) - 1) * 100

years = len(df_trade)

# Growth rates
cagr_exports = calc_cagr(df_trade['Exports'].iloc[0], df_trade['Exports'].iloc[-1], years-1)
cagr_reexports = calc_cagr(df_trade['Re-exports'].iloc[0], df_trade['Re-exports'].iloc[-1], years-1)
cagr_imports = calc_cagr(df_trade['Imports'].iloc[0], df_trade['Imports'].iloc[-1], years-1)
cagr_volume = calc_cagr(df_trade['Trade_Volume'].iloc[0], df_trade['Trade_Volume'].iloc[-1], years-1)

# Cumulative metrics
cum_exports = df_trade['Total_Exports'].sum()
cum_imports = df_trade['Imports'].sum()
cum_balance = df_trade['Trade_Balance'].sum()
cum_volume = df_trade['Trade_Volume'].sum()

# Balance profile
deficit_years = (df_trade['Trade_Balance'] < 0).sum()
surplus_years = (df_trade['Trade_Balance'] >= 0).sum()
peak_deficit = df_trade['Trade_Balance'].min()
peak_surplus = df_trade['Trade_Balance'].max()

# Coverage stats
avg_coverage = df_trade['Coverage_Ratio'].mean()
latest_coverage = df_trade['Coverage_Ratio'].iloc[-1]

# YoY volatility
yoy_exports = df_trade['Exports'].pct_change() * 100
yoy_imports = df_trade['Imports'].pct_change() * 100
vol_exports = yoy_exports.std()
vol_imports = yoy_imports.std()

# Re-export share
reexport_share = (df_trade['Re-exports'].sum() / df_trade['Total_Exports'].sum()) * 100

# Linear regression for trends
years_array = np.arange(len(df_trade)).reshape(-1, 1)
export_model = LinearRegression().fit(years_array, df_trade['Exports'].values)
import_model = LinearRegression().fit(years_array, df_trade['Imports'].values)

export_r2 = export_model.score(years_array, df_trade['Exports'].values)
import_r2 = import_model.score(years_array, df_trade['Imports'].values)

print("✅ All metrics calculated!")

# Display comprehensive summary
print("\n" + "="*80)
print("COMPREHENSIVE METRICS SUMMARY")
print("="*80)
print(f"\n{'CUMULATIVE (2015-2023):'}")
print(f"  Trade Volume:        ${cum_volume/1e9:>8.1f}B")
print(f"  Total Exports:       ${cum_exports/1e9:>8.1f}B")
print(f"  Total Imports:       ${cum_imports/1e9:>8.1f}B")
print(f"  Trade Balance:       ${cum_balance/1e9:>8.1f}B")

print(f"\n{'GROWTH RATES (CAGR):'}")
print(f"  Exports:             {cagr_exports:>7.1f}%")
print(f"  Re-exports:          {cagr_reexports:>7.1f}%")
print(f"  Imports:             {cagr_imports:>7.1f}%")
print(f"  Trade Volume:        {cagr_volume:>7.1f}%")

print(f"\n{'TRADE BALANCE PROFILE:'}")
print(f"  Deficit Years:       {deficit_years:>7} / {years}")
print(f"  Surplus Years:       {surplus_years:>7} / {years}")
print(f"  Peak Deficit:        ${abs(peak_deficit)/1e9:>7.1f}B")
print(f"  Peak Surplus:        ${peak_surplus/1e9:>7.1f}B")

print(f"\n{'COVERAGE & DEPENDENCY:'}")
print(f"  Latest Coverage:     {latest_coverage:>7.1f}%")
print(f"  Average Coverage:    {avg_coverage:>7.1f}%")
print(f"  Re-export Share:     {reexport_share:>7.1f}%")

print(f"\n{'VOLATILITY & TRENDS:'}")
print(f"  Export Volatility:   {vol_exports:>7.1f}%")
print(f"  Import Volatility:   {vol_imports:>7.1f}%")
print(f"  Export Trend R²:     {export_r2:>7.3f}")
print(f"  Import Trend R²:     {import_r2:>7.3f}")

if has_product_data and not bcg_export.empty:
    print(f"\n{'PRODUCT CONCENTRATION:'}")
    print(f"  Export HHI:          {hhi_exports:>7.0f}")
    print(f"  Import HHI:          {hhi_imports:>7.0f}")

    print(f"\n{'BCG PORTFOLIO (Exports):'}")
    if not bcg_summary.empty:
        for category in ['Star', 'Cash Cow', 'Question Mark', 'Dog']:
            if category in bcg_summary.index:
                count = bcg_summary.loc[category, 'Product']
                value = bcg_summary.loc[category, 'Total_Value']
                print(f"  {category + ':':<18} {count:>3.0f} products (${value/1e9:>6.2f}B)")

print("="*80)


📊 CALCULATING COMPREHENSIVE METRICS
✅ All metrics calculated!

COMPREHENSIVE METRICS SUMMARY

CUMULATIVE (2015-2023):
  Trade Volume:        $    37.6B
  Total Exports:       $    17.3B
  Total Imports:       $    20.4B
  Trade Balance:       $    -3.1B

GROWTH RATES (CAGR):
  Exports:               -27.5%
  Re-exports:             -2.9%
  Imports:                12.7%
  Trade Volume:            4.6%

TRADE BALANCE PROFILE:
  Deficit Years:             3 / 10
  Surplus Years:             7 / 10
  Peak Deficit:        $    3.2B
  Peak Surplus:        $    0.9B

COVERAGE & DEPENDENCY:
  Latest Coverage:         7.9%
  Average Coverage:      123.0%
  Re-export Share:        13.0%

VOLATILITY & TRENDS:
  Export Volatility:      45.5%
  Import Volatility:      75.7%
  Export Trend R²:       0.072
  Import Trend R²:       0.624


In [50]:
# ============================================================================
# CELL 9: GENERATE ALL VISUALIZATIONS (HIGH QUALITY)
# ============================================================================

print("\n📊 GENERATING COMPREHENSIVE VISUALIZATIONS")
print("="*80)

years_list = [int(y) for y in df_trade.index]

# ========================================================================
# FIGURE 1: TRADE FLOW TRENDS
# ========================================================================
fig1, ax1 = plt.subplots(figsize=(14, 8))
ax1.plot(years_list, df_trade['Exports']/1e9, marker='o', linewidth=3, markersize=10,
         label='Direct Exports', color='#2E86AB')
ax1.plot(years_list, df_trade['Re-exports']/1e9, marker='s', linewidth=3, markersize=10,
         label='Re-exports', color='#A23B72')
ax1.plot(years_list, df_trade['Imports']/1e9, marker='^', linewidth=3, markersize=10,
         label='Imports', color='#F18F01')
ax1.plot(years_list, df_trade['Total_Exports']/1e9, marker='D', linewidth=2.5,
         linestyle='--', label='Total Exports', color='#06A77D')

# Add trend lines
X = np.arange(len(years_list)).reshape(-1, 1)
for col, color, label in [('Exports', '#2E86AB', 'Export Trend'),
                           ('Imports', '#F18F01', 'Import Trend')]:
    y = df_trade[col].values / 1e9
    model = LinearRegression().fit(X, y)
    trend = model.predict(X)
    ax1.plot(years_list, trend, linestyle=':', linewidth=2, color=color, alpha=0.5)

ax1.set_xlabel('Year', fontsize=13, fontweight='bold')
ax1.set_ylabel('Trade Value (USD Billions)', fontsize=13, fontweight='bold')
ax1.set_title('Oman-KSA Trade Flow Trends (2015-2023)', fontsize=16, fontweight='bold', pad=20)
ax1.legend(loc='best', frameon=True, shadow=True, fontsize=11)
ax1.grid(True, alpha=0.3)
ax1.set_xticks(years_list)
plt.tight_layout()
fig1.savefig('/tmp/fig1_trade_flows.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Figure 1: Trade Flow Trends")

# ========================================================================
# FIGURE 2: TRADE BALANCE EVOLUTION
# ========================================================================
fig2, ax2 = plt.subplots(figsize=(14, 8))
balance_values = df_trade['Trade_Balance']/1e9
colors = ['#06A77D' if x >= 0 else '#C73E1D' for x in balance_values]

bars = ax2.bar(years_list, balance_values, color=colors, alpha=0.7,
               edgecolor='black', linewidth=1.5, width=0.6)
ax2.axhline(y=0, color='black', linestyle='-', linewidth=2)

# Add value labels
for year, val in zip(years_list, balance_values):
    ax2.text(year, val + (0.3 if val >= 0 else -0.3), f'${val:.1f}B',
            ha='center', va='bottom' if val >= 0 else 'top',
            fontsize=10, fontweight='bold')

ax2.set_xlabel('Year', fontsize=13, fontweight='bold')
ax2.set_ylabel('Trade Balance (USD Billions)', fontsize=13, fontweight='bold')
ax2.set_title('Oman-KSA Trade Balance Evolution (Including Re-exports)',
             fontsize=16, fontweight='bold', pad=20)
ax2.grid(True, alpha=0.3, axis='y')
ax2.set_xticks(years_list)

# Add annotation
avg_balance = balance_values.mean()
ax2.text(0.98, 0.02, f'Average Balance: ${avg_balance:.1f}B',
        transform=ax2.transAxes, ha='right', va='bottom',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
        fontsize=10, fontweight='bold')

plt.tight_layout()
fig2.savefig('/tmp/fig2_balance.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Figure 2: Trade Balance Evolution")

# ========================================================================
# FIGURE 3: COVERAGE RATIO TREND
# ========================================================================
fig3, ax3 = plt.subplots(figsize=(14, 8))
coverage = df_trade['Coverage_Ratio']

ax3.plot(years_list, coverage, marker='o', linewidth=3, color='#2E86AB', markersize=10)
ax3.axhline(y=100, color='red', linestyle='--', linewidth=2.5, label='Equilibrium (100%)')
ax3.fill_between(years_list, coverage, 100, where=(coverage < 100),
                 alpha=0.25, color='#C73E1D', label='Deficit Zone')
ax3.fill_between(years_list, coverage, 100, where=(coverage >= 100),
                 alpha=0.25, color='#06A77D', label='Surplus Zone')

# Add value labels
for year, val in zip(years_list, coverage):
    # Only add label if value is finite
    if np.isfinite(val):
        ax3.text(year, val + 3, f'{val:.1f}%', ha='center', va='bottom',
                fontsize=10, fontweight='bold')

ax3.set_xlabel('Year', fontsize=13, fontweight='bold')
ax3.set_ylabel('Coverage Ratio (%)', fontsize=13, fontweight='bold')
ax3.set_title('Oman Export Coverage of Imports from KSA',
             fontsize=16, fontweight='bold', pad=20)
ax3.legend(loc='best', frameon=True, shadow=True, fontsize=11)
ax3.grid(True, alpha=0.3)
ax3.set_xticks(years_list)

# --- FIX START ---
# Filter out infinite values for calculating plot limits
finite_coverage = coverage[np.isfinite(coverage)]

if not finite_coverage.empty:
    min_val = finite_coverage.min()
    max_val = finite_coverage.max()
    # Ensure lower limit is not negative if min_val is small
    bottom_limit = max(0, min_val - 10)
    top_limit = max_val + 15
    ax3.set_ylim([bottom_limit, top_limit])
else:
    # If all coverage values are infinite (e.g., imports are zero for all years),
    # set a default reasonable range and add a warning.
    ax3.set_ylim([0, 200]) # Default reasonable range for coverage ratio
    ax3.text(0.5, 0.5, "Coverage Ratio data is infinite or empty (Imports are zero).", transform=ax3.transAxes,
             ha='center', va='center', fontsize=12, color='red', bbox=dict(facecolor='white', alpha=0.8))
    print("  ⚠️  Warning: Coverage Ratio is infinite for all years (Imports are zero). Setting default Y-axis limits for plot 3.")
# --- FIX END ---

plt.tight_layout()
fig3.savefig('/tmp/fig3_coverage.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Figure 3: Coverage Ratio Trend")

# ========================================================================
# FIGURE 4: CAGR COMPARISON
# ========================================================================
fig4, ax4 = plt.subplots(figsize=(14, 8))
categories = ['Direct Exports', 'Re-exports', 'Imports', 'Total Exports', 'Trade Volume']
cagrs = [cagr_exports, cagr_reexports, cagr_imports,
         calc_cagr(df_trade['Total_Exports'].iloc[0], df_trade['Total_Exports'].iloc[-1], years-1),
         cagr_volume]
colors_bar = ['#2E86AB', '#A23B72', '#F18F01', '#06A77D', '#DD1C1A']

bars = ax4.barh(categories, cagrs, color=colors_bar, alpha=0.8,
                edgecolor='black', linewidth=1.5, height=0.6)
ax4.axvline(x=0, color='black', linestyle='-', linewidth=2)

# Add value labels
for i, (cat, val) in enumerate(zip(categories, cagrs)):
    ax4.text(val + (0.4 if val >= 0 else -0.4), i, f'{val:.1f}%',
            va='center', ha='left' if val >= 0 else 'right',
            fontsize=11, fontweight='bold')

ax4.set_xlabel('Compound Annual Growth Rate (%)', fontsize=13, fontweight='bold')
ax4.set_title('CAGR Analysis (2015-2023)', fontsize=16, fontweight='bold', pad=20)
ax4.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
fig4.savefig('/tmp/fig4_cagr.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Figure 4: CAGR Comparison")

# ========================================================================
# FIGURE 5: EXPORT COMPOSITION (STACKED AREA)
# ========================================================================
fig5, ax5 = plt.subplots(figsize=(14, 8))

ax5.fill_between(years_list, 0, df_trade['Exports']/1e9,
                 alpha=0.7, color='#2E86AB', label='Direct Exports')
ax5.fill_between(years_list, df_trade['Exports']/1e9, df_trade['Total_Exports']/1e9,
                 alpha=0.7, color='#A23B72', label='Re-exports')

# Add percentage labels
for year in years_list:
    total = df_trade.loc[str(year), 'Total_Exports']
    reexp = df_trade.loc[str(year), 'Re-exports']
    pct = (reexp / total * 100) if total > 0 else 0
    y_pos = df_trade.loc[str(year), 'Total_Exports']/1e9 + 0.5
    ax5.text(year, y_pos, f'{pct:.1f}%\nRe-exp', ha='center',
            fontsize=8, fontweight='bold')

ax5.set_xlabel('Year', fontsize=13, fontweight='bold')
ax5.set_ylabel('Export Value (USD Billions)', fontsize=13, fontweight='bold')
ax5.set_title('Oman Export Composition to KSA', fontsize=16, fontweight='bold', pad=20)
ax5.legend(loc='upper left', frameon=True, shadow=True, fontsize=11)
ax5.grid(True, alpha=0.3, axis='y')
ax5.set_xticks(years_list)

plt.tight_layout()
fig5.savefig('/tmp/fig5_composition.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Figure 5: Export Composition")

# ========================================================================
# FIGURE 6: BCG MATRIX (IF AVAILABLE)
# ========================================================================
if has_product_data and not bcg_export.empty:
    fig6, ax6 = plt.subplots(figsize=(14, 10))

    # Plot each category with different colors
    colors_bcg = {
        'Star': '#FFD700',
        'Cash Cow': '#90EE90',
        'Question Mark': '#FF6B6B',
        'Dog': '#D3D3D3'
    }

    for category in ['Star', 'Cash Cow', 'Question Mark', 'Dog']:
        data = bcg_export[bcg_export['BCG_Category'] == category]
        if not data.empty:
            ax6.scatter(data['Market_Share'], data['CAGR'],
                       s=data['Total_Value']/1e6,  # Size by value
                       c=colors_bcg[category], alpha=0.6, edgecolors='black',
                       linewidth=1.5, label=category)

    # Add median lines
    ax6.axvline(x=bcg_export['Market_Share'].median(), color='black',
               linestyle='--', linewidth=2, alpha=0.5)
    ax6.axhline(y=bcg_export['CAGR'].median(), color='black',
               linestyle='--', linewidth=2, alpha=0.5)

    # Add quadrant labels
    ax6.text(0.02, 0.98, 'Question Mark', transform=ax6.transAxes,
            fontsize=12, fontweight='bold', va='top')
    ax6.text(0.98, 0.98, 'Star', transform=ax6.transAxes,
            fontsize=12, fontweight='bold', va='top', ha='right')
    ax6.text(0.02, 0.02, 'Dog', transform=ax6.transAxes,
            fontsize=12, fontweight='bold', va='bottom')
    ax6.text(0.98, 0.02, 'Cash Cow', transform=ax6.transAxes,
            fontsize=12, fontweight='bold', va='bottom', ha='right')

    ax6.set_xlabel('Market Share (%)', fontsize=13, fontweight='bold')
    ax6.set_ylabel('CAGR (%)', fontsize=13, fontweight='bold')
    ax6.set_title('BCG Portfolio Matrix - Oman Exports to KSA',
                 fontsize=16, fontweight='bold', pad=20)
    ax6.legend(loc='center left', bbox_to_anchor=(1, 0.5), frameon=True, shadow=True)
    ax6.grid(True, alpha=0.3)

    plt.tight_layout()
    fig6.savefig('/tmp/fig6_bcg_matrix.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✅ Figure 6: BCG Matrix")

# ========================================================================
# FIGURE 7: YEAR-OVER-YEAR GROWTH RATES
# ========================================================================
fig7, ax7 = plt.subplots(figsize=(14, 8))

yoy_exp = df_trade['Exports'].pct_change() * 100
yoy_imp = df_trade['Imports'].pct_change() * 100

ax7.plot(years_list[1:], yoy_exp.iloc[1:], marker='o', linewidth=2.5,
         markersize=8, label='Exports YoY%', color='#2E86AB')
ax7.plot(years_list[1:], yoy_imp.iloc[1:], marker='^', linewidth=2.5,
         markersize=8, label='Imports YoY%', color='#F18F01')
ax7.axhline(y=0, color='black', linestyle='--', linewidth=1.5)

ax7.set_xlabel('Year', fontsize=13, fontweight='bold')
ax7.set_ylabel('Year-over-Year Growth (%)', fontsize=13, fontweight='bold')
ax7.set_title('Annual Growth Rate Dynamics', fontsize=16, fontweight='bold', pad=20)
ax7.legend(loc='best', frameon=True, shadow=True, fontsize=11)
ax7.grid(True, alpha=0.3)
ax7.set_xticks(years_list[1:])

plt.tight_layout()
fig7.savefig('/tmp/fig7_yoy_growth.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Figure 7: YoY Growth Rates")

print("\n✅ ALL VISUALIZATIONS GENERATED!")
print("="*80)


📊 GENERATING COMPREHENSIVE VISUALIZATIONS
✅ Figure 1: Trade Flow Trends
✅ Figure 2: Trade Balance Evolution
✅ Figure 3: Coverage Ratio Trend
✅ Figure 4: CAGR Comparison
✅ Figure 5: Export Composition
✅ Figure 7: YoY Growth Rates

✅ ALL VISUALIZATIONS GENERATED!


In [51]:
# ============================================================================
# CELL 10: GENERATE COMPREHENSIVE WORD DOCUMENT
# ============================================================================

print("\n📄 GENERATING COMPREHENSIVE WORD DOCUMENT REPORT")
print("="*80)

REPORTING_COUNTRY = 'Oman'
PARTNER_COUNTRY = 'KSA'

doc = Document()

# ========================================================================
# CONFIGURE STYLES
# ========================================================================
styles = doc.styles

# Title style
title_style = styles['Title']
title_style.font.size = Pt(26)
title_style.font.bold = True
title_style.font.color.rgb = RGBColor(31, 78, 120)

# Heading styles
h1_style = styles['Heading 1']
h1_style.font.size = Pt(18)
h1_style.font.bold = True
h1_style.font.color.rgb = RGBColor(31, 78, 120)

h2_style = styles['Heading 2']
h2_style.font.size = Pt(15)
h2_style.font.bold = True
h2_style.font.color.rgb = RGBColor(46, 116, 181)

# ========================================================================
# TITLE PAGE
# ========================================================================
print("  Creating title page...")

title = doc.add_heading(f'{REPORTING_COUNTRY}-{PARTNER_COUNTRY}', level=0)
title.alignment = WD_ALIGN_PARAGRAPH.CENTER

subtitle = doc.add_heading('Comprehensive Bilateral Trade Dynamics Report', level=2)
subtitle.alignment = WD_ALIGN_PARAGRAPH.CENTER

subtitle2 = doc.add_heading('Strategic Assessment & Portfolio Analysis', level=3)
subtitle2.alignment = WD_ALIGN_PARAGRAPH.CENTER

doc.add_paragraph()
period = doc.add_paragraph(f'Analysis Period: 2015-2023')
period.alignment = WD_ALIGN_PARAGRAPH.CENTER
period.runs[0].font.size = Pt(12)

date_para = doc.add_paragraph(f'Report Date: {datetime.now().strftime("%B %d, %Y")}')
date_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
date_para.runs[0].font.size = Pt(12)

doc.add_paragraph()
doc.add_paragraph()
doc.add_paragraph()

prepared = doc.add_paragraph('Prepared by:')
prepared.alignment = WD_ALIGN_PARAGRAPH.CENTER
prepared.runs[0].font.size = Pt(11)

company = doc.add_paragraph('Deloitte Consulting - GCC Trade Analytics Division')
company.alignment = WD_ALIGN_PARAGRAPH.CENTER
company.runs[0].font.bold = True
company.runs[0].font.size = Pt(12)

doc.add_page_break()

# ========================================================================
# EXECUTIVE SUMMARY
# ========================================================================
print("  Creating executive summary...")

doc.add_heading('Executive Summary', level=1)

# Narrative
exec_narrative = f"""
The bilateral trade relationship between {REPORTING_COUNTRY} and {PARTNER_COUNTRY} over the nine-year period (2015-2023)
generated a cumulative trade volume of ${cum_volume/1e9:.1f} billion USD, with a cumulative trade balance of
${cum_balance/1e9:.1f} billion USD. The relationship is characterized by a structural trade imbalance, recording
deficits in {deficit_years} out of {years} years analyzed.

Key characteristics include:
- Trade volume grew at {cagr_volume:.1f}% CAGR, indicating {'robust' if cagr_volume > 5 else 'moderate'} bilateral economic integration
- Exports expanded at {cagr_exports:.1f}% CAGR, {'outpacing' if cagr_exports > cagr_imports else 'lagging'} import growth ({cagr_imports:.1f}% CAGR)
- Export coverage averaged {avg_coverage:.1f}%, reflecting {'significant' if avg_coverage < 60 else 'moderate'} import dependency
- Re-exports constitute {reexport_share:.1f}% of total exports, underscoring {REPORTING_COUNTRY}'s {'significant' if reexport_share > 20 else 'limited'} role as a regional trade hub
"""

doc.add_paragraph(exec_narrative)

# Key Metrics Table
doc.add_heading('Key Performance Indicators', level=2)

metrics_table = doc.add_table(rows=1, cols=2)
metrics_table.style = 'Light Grid Accent 1'
hdr = metrics_table.rows[0].cells
hdr[0].text = 'Metric'
hdr[1].text = 'Value'
hdr[0].paragraphs[0].runs[0].font.bold = True
hdr[1].paragraphs[0].runs[0].font.bold = True

key_metrics = [
    ('Cumulative Trade Volume (2015-2023)', f'${cum_volume/1e9:.1f}B'),
    ('Cumulative Trade Balance', f'${cum_balance/1e9:.1f}B'),
    ('Trade Volume CAGR', f'{cagr_volume:.1f}%'),
    ('Export CAGR', f'{cagr_exports:.1f}%'),
    ('Import CAGR', f'{cagr_imports:.1f}%'),
    ('Re-export CAGR', f'{cagr_reexports:.1f}%'),
    ('Latest Year Coverage Ratio', f'{latest_coverage:.1f}%'),
    ('Average Coverage Ratio', f'{avg_coverage:.1f}%'),
    ('Deficit Years', f'{deficit_years} out of {years}'),
    ('Peak Deficit', f'${abs(peak_deficit)/1e9:.1f}B'),
    ('Re-export Share of Total Exports', f'{reexport_share:.1f}%'),
    ('Export Volatility (Std Dev)', f'{vol_exports:.1f}%'),
    ('Import Volatility (Std Dev)', f'{vol_imports:.1f}%'),
]

if has_product_data and hhi_exports > 0:
    key_metrics.extend([
        ('Export HHI Concentration Index', f'{hhi_exports:.0f}'),
        ('Import HHI Concentration Index', f'{hhi_imports:.0f}'),
    ])

for metric, value in key_metrics:
    row = metrics_table.add_row().cells
    row[0].text = metric
    row[1].text = value

doc.add_page_break()

# ========================================================================
# SECTION 1: TRADE BALANCE ANALYSIS
# ========================================================================
print("  Creating Section 1: Trade Balance Analysis...")

doc.add_heading('1. Trade Balance Analysis and Structural Assessment', level=1)

balance_text = f"""
The bilateral trade relationship exhibits a persistent deficit pattern, with cumulative trade balance of
${cum_balance/1e9:.1f}B over the {years}-year period. This structural imbalance reflects fundamental asymmetries
in the economic relationship, with {PARTNER_COUNTRY} serving as a major supplier to the {REPORTING_COUNTRY} market
while {REPORTING_COUNTRY} faces challenges in penetrating the {PARTNER_COUNTRY} market at comparable scale.
"""
doc.add_paragraph(balance_text)

doc.add_heading('1.1 Annual Trade Balance Dynamics', level=2)

# Annual balance table
balance_table = doc.add_table(rows=1, cols=7)
balance_table.style = 'Light Grid Accent 1'
hdr = balance_table.rows[0].cells
headers = ['Year', 'Exports\n($M)', 'Re-exports\n($M)', 'Imports\n($M)',
           'Total Exports\n($M)', 'Balance\n($M)', 'Coverage\n(%)']
for i, h in enumerate(headers):
    hdr[i].text = h
    hdr[i].paragraphs[0].runs[0].font.bold = True

for year, row_data in df_trade.iterrows():
    cells = balance_table.add_row().cells
    cells[0].text = str(year)
    cells[1].text = f"{row_data['Exports']/1e6:,.0f}"
    cells[2].text = f"{row_data['Re-exports']/1e6:,.0f}"
    cells[3].text = f"{row_data['Imports']/1e6:,.0f}"
    cells[4].text = f"{row_data['Total_Exports']/1e6:,.0f}"
    cells[5].text = f"{row_data['Trade_Balance']/1e6:,.0f}"
    cells[6].text = f"{row_data['Coverage_Ratio']:.1f}"

doc.add_paragraph()

# Key insights
doc.add_heading('1.2 Key Balance Insights', level=2)

insights = [
    f"Persistent deficit pattern: {deficit_years} out of {years} years recorded trade deficits",
    f"Peak deficit of ${abs(peak_deficit)/1e9:.1f}B occurred in {df_trade['Trade_Balance'].idxmin()}",
    f"{'Improving trend' if df_trade['Trade_Balance'].iloc[-1] > df_trade['Trade_Balance'].iloc[0] else 'Widening deficit'} observed over the analysis period",
    f"Cumulative deficit represents {abs(cum_balance)/cum_volume*100:.1f}% of total trade volume",
]

for insight in insights:
    p = doc.add_paragraph(insight, style='List Bullet')

doc.add_paragraph()
doc.add_heading('1.3 Trade Flow Visualization', level=2)
doc.add_picture('/tmp/fig1_trade_flows.png', width=Inches(6.5))
doc.add_paragraph('Figure 1: Trade flow trends showing exports, re-exports, and imports with linear trend lines (2015-2023).',
                 style='Caption')

doc.add_page_break()

# ========================================================================
# SECTION 2: GROWTH DYNAMICS
# ========================================================================
print("  Creating Section 2: Growth Dynamics...")

doc.add_heading('2. Growth Dynamics and Time Series Analysis', level=1)

growth_narrative = f"""
The bilateral trade relationship demonstrates dynamic growth patterns with differentiated trajectories across trade components.
Direct exports from {REPORTING_COUNTRY} to {PARTNER_COUNTRY} grew at {cagr_exports:.1f}% CAGR,
{'demonstrating expanding market penetration and competitive positioning' if cagr_exports > 0 else 'indicating market share challenges and competitive pressures'}.
Re-exports contributed {reexport_share:.1f}% of total export value, growing at {cagr_reexports:.1f}% CAGR, which underscores
{REPORTING_COUNTRY}'s {'significant' if reexport_share > 20 else 'limited'} role as a regional entrepôt center.

Import growth at {cagr_imports:.1f}% CAGR {'exceeded export expansion' if cagr_imports > cagr_exports else 'trailed export growth'},
contributing to {'widening' if cagr_imports > cagr_exports else 'narrowing'} trade imbalances. The overall trade volume expanded at
{cagr_volume:.1f}% CAGR, reflecting {'deepening' if cagr_volume > 5 else 'moderate'} economic integration within the GCC framework.
"""
doc.add_paragraph(growth_narrative)

doc.add_heading('2.1 Compound Annual Growth Rates', level=2)
doc.add_picture('/tmp/fig4_cagr.png', width=Inches(6.5))
doc.add_paragraph('Figure 2: CAGR comparison across all trade components (2015-2023).', style='Caption')

doc.add_paragraph()
doc.add_heading('2.2 Trend Analysis and Statistical Significance', level=2)

trend_text = f"""
Linear regression analysis reveals:
- Export trend R² = {export_r2:.3f} ({'strong' if export_r2 > 0.7 else 'moderate' if export_r2 > 0.4 else 'weak'} trend fit)
- Import trend R² = {import_r2:.3f} ({'strong' if import_r2 > 0.7 else 'moderate' if import_r2 > 0.4 else 'weak'} trend fit)

The {'high' if max(export_r2, import_r2) > 0.7 else 'moderate'} R² values indicate {'predictable' if max(export_r2, import_r2) > 0.7 else 'somewhat volatile'}
trade patterns with {'minimal' if max(export_r2, import_r2) > 0.7 else 'notable'} deviation from linear trends.
"""
doc.add_paragraph(trend_text)

doc.add_heading('2.3 Volatility Assessment', level=2)

# Volatility table
vol_table = doc.add_table(rows=1, cols=4)
vol_table.style = 'Light Grid Accent 1'
vol_hdr = vol_table.rows[0].cells
vol_headers = ['Component', 'YoY Volatility (%)', 'Assessment', 'Interpretation']
for i, h in enumerate(vol_headers):
    vol_hdr[i].text = h
    vol_hdr[i].paragraphs[0].runs[0].font.bold = True

vol_data = [
    ('Exports', vol_exports,
     'High' if vol_exports > 20 else 'Moderate' if vol_exports > 10 else 'Low',
     'Significant fluctuations' if vol_exports > 20 else 'Stable growth' if vol_exports < 10 else 'Some variability'),
    ('Imports', vol_imports,
     'High' if vol_imports > 20 else 'Moderate' if vol_imports > 10 else 'Low',
     'Significant fluctuations' if vol_imports > 20 else 'Stable growth' if vol_imports < 10 else 'Some variability'),
]

for comp, vol, assess, interp in vol_data:
    row = vol_table.add_row().cells
    row[0].text = comp
    row[1].text = f'{vol:.1f}%'
    row[2].text = assess
    row[3].text = interp

doc.add_paragraph()
doc.add_heading('2.4 Year-over-Year Growth Dynamics', level=2)
doc.add_picture('/tmp/fig7_yoy_growth.png', width=Inches(6.5))
doc.add_paragraph('Figure 3: Annual growth rate fluctuations showing volatility patterns.', style='Caption')

doc.add_page_break()

# ========================================================================
# SECTION 3: TRADE BALANCE DECOMPOSITION
# ========================================================================
print("  Creating Section 3: Trade Balance Decomposition...")

doc.add_heading('3. Trade Balance Decomposition and Coverage Analysis', level=1)

doc.add_heading('3.1 Trade Balance Evolution', level=2)
doc.add_picture('/tmp/fig2_balance.png', width=Inches(6.5))
doc.add_paragraph('Figure 4: Annual trade balance showing persistent deficit pattern with value annotations.', style='Caption')

doc.add_paragraph()

deficit_analysis = f"""
The trade balance exhibits a {'persistent' if deficit_years >= years * 0.7 else 'intermittent'} deficit pattern.
The cumulative deficit of ${abs(cum_balance)/1e9:.1f}B represents {abs(cum_balance)/cum_volume*100:.1f}% of total bilateral trade volume,
indicating structural imbalances rather than cyclical fluctuations.

Key observations:
- Average annual deficit: ${abs(df_trade['Trade_Balance'].mean())/1e9:.1f}B
- Peak deficit year: {df_trade['Trade_Balance'].idxmin()} (${abs(peak_deficit)/1e9:.1f}B)
- {'Deteriorating' if df_trade['Trade_Balance'].iloc[-1] < df_trade['Trade_Balance'].iloc[0] else 'Improving'} trend in recent years
"""
doc.add_paragraph(deficit_analysis)

doc.add_heading('3.2 Export Coverage of Imports', level=2)
doc.add_picture('/tmp/fig3_coverage.png', width=Inches(6.5))
doc.add_paragraph('Figure 5: Export coverage ratio showing the extent of import dependency.', style='Caption')

doc.add_paragraph()

coverage_analysis = f"""
The export coverage ratio averaged {avg_coverage:.1f}% over the analysis period, with the latest year at {latest_coverage:.1f}%.
This indicates that {REPORTING_COUNTRY}'s total exports (including re-exports) cover only {latest_coverage:.0f}% of its import bill
from {PARTNER_COUNTRY}, reflecting {'high' if latest_coverage < 50 else 'moderate' if latest_coverage < 75 else 'low'} import dependency.

Coverage ratio insights:
- Minimum coverage: {df_trade['Coverage_Ratio'].min():.1f}% in {df_trade['Coverage_Ratio'].idxmin()}
- Maximum coverage: {df_trade['Coverage_Ratio'].max():.1f}% in {df_trade['Coverage_Ratio'].idxmax()}
- {'Improving' if df_trade['Coverage_Ratio'].iloc[-1] > df_trade['Coverage_Ratio'].iloc[0] else 'Declining'} trend observed
- {latest_coverage:.0f}% coverage implies ${(100-latest_coverage)/100 * df_trade['Imports'].iloc[-1]/1e9:.1f}B uncovered import demand
"""
doc.add_paragraph(coverage_analysis)

doc.add_heading('3.3 Re-export Trade Dynamics', level=2)
doc.add_picture('/tmp/fig5_composition.png', width=Inches(6.5))
doc.add_paragraph('Figure 6: Export composition showing the contribution of direct exports versus re-exports.', style='Caption')

doc.add_paragraph()

reexport_analysis = f"""
Re-exports constitute {reexport_share:.1f}% of total exports to {PARTNER_COUNTRY}, growing at {cagr_reexports:.1f}% CAGR.
This {'substantial' if reexport_share > 20 else 'moderate' if reexport_share > 10 else 'limited'} re-export activity underscores
{REPORTING_COUNTRY}'s role as {'a significant regional trade hub' if reexport_share > 20 else 'a limited entrepôt center'} for goods
destined to {PARTNER_COUNTRY}.

Re-export trends:
- Average re-export share: {(df_trade['Re-exports'] / df_trade['Total_Exports'] * 100).mean():.1f}%
- Re-export peak year: {df_trade['Re-exports'].idxmax()} (${df_trade['Re-exports'].max()/1e9:.1f}B)
- {'Increasing' if df_trade['Re-exports'].iloc[-1] > df_trade['Re-exports'].iloc[0] else 'Decreasing'} re-export activity over time
"""
doc.add_paragraph(reexport_analysis)

doc.add_page_break()

# ========================================================================
# SECTION 4: PRODUCT PORTFOLIO ANALYSIS (IF AVAILABLE)
# ========================================================================
if has_product_data and not bcg_export.empty:
    print("  Creating Section 4: Product Portfolio Analysis...")

    doc.add_heading('4. Product Portfolio and Concentration Analysis', level=1)

    doc.add_heading('4.1 BCG Matrix Portfolio Assessment', level=2)

    bcg_narrative = f"""
The BCG Matrix analysis categorizes {len(bcg_export)} export products based on market share and growth rates,
providing strategic insights for portfolio optimization and resource allocation.
"""
    doc.add_paragraph(bcg_narrative)

    # BCG Summary Table
    bcg_table = doc.add_table(rows=1, cols=4)
    bcg_table.style = 'Light Grid Accent 1'
    bcg_hdr = bcg_table.rows[0].cells
    bcg_headers = ['Category', 'Product Count', 'Total Value ($B)', 'Avg CAGR (%)']
    for i, h in enumerate(bcg_headers):
        bcg_hdr[i].text = h
        bcg_hdr[i].paragraphs[0].runs[0].font.bold = True

    if not bcg_summary.empty:
        for category in ['Star', 'Cash Cow', 'Question Mark', 'Dog']:
            if category in bcg_summary.index:
                row = bcg_table.add_row().cells
                row[0].text = category
                row[1].text = f"{bcg_summary.loc[category, 'Product']:.0f}"
                row[2].text = f"${bcg_summary.loc[category, 'Total_Value']/1e9:.2f}"
                row[3].text = f"{bcg_summary.loc[category, 'CAGR']:.1f}%"

    doc.add_paragraph()
    doc.add_picture('/tmp/fig6_bcg_matrix.png', width=Inches(6.5))
    doc.add_paragraph('Figure 7: BCG Portfolio Matrix showing product distribution across strategic categories.', style='Caption')

    doc.add_paragraph()
    doc.add_heading('4.2 Star Products (High Growth, High Share)', level=2)

    if 'Star' in bcg_export['BCG_Category'].values:
        stars = bcg_export[bcg_export['BCG_Category'] == 'Star'].nlargest(10, 'Total_Value')

        star_table = doc.add_table(rows=1, cols=4)
        star_table.style = 'Light Grid Accent 1'
        star_hdr = star_table.rows[0].cells
        star_headers = ['Product', 'CAGR (%)', 'Market Share (%)', 'Value ($M)']
        for i, h in enumerate(star_headers):
            star_hdr[i].text = h
            star_hdr[i].paragraphs[0].runs[0].font.bold = True

        for idx, row in stars.head(10).iterrows():
            cells = star_table.add_row().cells
            cells[0].text = str(row['Product'])[:50]
            cells[1].text = f"{row['CAGR']:.1f}"
            cells[2].text = f"{row['Market_Share']:.2f}"
            cells[3].text = f"${row['Total_Value']/1e6:.1f}"

        doc.add_paragraph()
        star_text = f"""
Star products represent high-growth, high-market-share items requiring continued investment to maintain competitive advantage.
These {len(stars)} products collectively account for ${stars['Total_Value'].sum()/1e9:.2f}B in export value.
"""
        doc.add_paragraph(star_text)

    doc.add_heading('4.3 Concentration Analysis (HHI)', level=2)

    hhi_text = f"""
The Herfindahl-Hirschman Index (HHI) measures market concentration:

Export Market Concentration:
- HHI: {hhi_exports:.0f}
- Assessment: {'Low concentration (competitive market)' if hhi_exports < 1500 else 'Moderate concentration' if hhi_exports < 2500 else 'High concentration (oligopolistic)'}
- Top 10 products control {bcg_export.nlargest(10, 'Market_Share')['Market_Share'].sum():.1f}% of export value

Import Market Concentration:
- HHI: {hhi_imports:.0f}
- Assessment: {'Low concentration (competitive market)' if hhi_imports < 1500 else 'Moderate concentration' if hhi_imports < 2500 else 'High concentration (oligopolistic)'}
- Top 10 products control {bcg_import.nlargest(10, 'Market_Share')['Market_Share'].sum():.1f}% of import value

{'High concentration suggests vulnerability to supply disruptions and limited diversification' if max(hhi_exports, hhi_imports) > 2500 else 'Moderate concentration indicates balanced portfolio with manageable concentration risk' if max(hhi_exports, hhi_imports) > 1500 else 'Low concentration reflects a well-diversified trade portfolio'}.
"""
    doc.add_paragraph(hhi_text)

    doc.add_heading('4.4 Top Products by Value', level=2)

    # Top exports table
    top_exp_table = doc.add_table(rows=1, cols=3)
    top_exp_table.style = 'Light Grid Accent 1'
    top_exp_hdr = top_exp_table.rows[0].cells
    top_exp_hdr[0].text = 'Top Export Products'
    top_exp_hdr[1].text = 'Market Share (%)'
    top_exp_hdr[2].text = 'Value ($M)'
    for cell in top_exp_hdr:
        cell.paragraphs[0].runs[0].font.bold = True

    top_exports = bcg_export.nlargest(10, 'Market_Share')
    for idx, row in top_exports.iterrows():
        cells = top_exp_table.add_row().cells
        cells[0].text = str(row['Product'])[:60]
        cells[1].text = f"{row['Market_Share']:.2f}%"
        cells[2].text = f"${row['Total_Value']/1e6:.1f}"

    doc.add_page_break()

# ========================================================================
# SECTION 5: STRATEGIC RECOMMENDATIONS
# ========================================================================
print("  Creating Section 5: Strategic Recommendations...")

doc.add_heading('4. Strategic Recommendations and Policy Priorities' if not has_product_data else '5. Strategic Recommendations and Policy Priorities', level=1)

strategic_intro = f"""
Given the structural characteristics of the bilateral trade relationship—persistent deficits, {'high' if latest_coverage < 50 else 'moderate'} import dependency,
and {cagr_volume:.1f}% trade volume growth—{REPORTING_COUNTRY} should prioritize a multi-faceted strategy encompassing export expansion,
import substitution, and trade facilitation initiatives.
"""
doc.add_paragraph(strategic_intro)

doc.add_heading('4.1 Export Expansion Strategies' if not has_product_data else '5.1 Export Expansion Strategies', level=2)

if cagr_exports < cagr_imports:
    export_intro = f"""
Given that exports are growing slower than imports ({cagr_exports:.1f}% vs {cagr_imports:.1f}% CAGR), {REPORTING_COUNTRY} must
aggressively pursue export expansion through:
"""
else:
    export_intro = f"""
While exports are growing faster than imports ({cagr_exports:.1f}% vs {cagr_imports:.1f}% CAGR), {REPORTING_COUNTRY} should
consolidate gains and accelerate market penetration through:
"""
doc.add_paragraph(export_intro)

export_strategies = [
    f"Market Diversification: Expand beyond current product concentration (HHI: {hhi_exports:.0f}) to reduce vulnerability to demand shocks",
    f"Value Chain Upgrading: Move into higher value-added products to improve export unit values and margins",
    "Trade Facilitation: Reduce transaction costs and time-to-market through customs modernization and logistics optimization",
    f"Strategic Partnerships: Forge alliances with key distributors and importers in {PARTNER_COUNTRY} to enhance market access",
    "Export Financing: Establish dedicated credit facilities and export credit insurance to support SME exporters",
    "Quality Certification: Fast-track mutual recognition agreements for standards and certifications to reduce market entry barriers",
    f"Sector Focus: {'Target Star products for investment and Question Marks for growth acceleration' if has_product_data else 'Identify high-potential sectors through market research'}",
]

for strategy in export_strategies:
    doc.add_paragraph(strategy, style='List Bullet')

doc.add_heading('4.2 Import Substitution and Localization' if not has_product_data else '5.2 Import Substitution and Localization', level=2)

import_sub_intro = f"""
With imports at ${df_trade['Imports'].iloc[-1]/1e9:.1f}B annually (latest year) and growing at {cagr_imports:.1f}% CAGR,
strategic import substitution opportunities should be evaluated in sectors where:
"""
doc.add_paragraph(import_sub_intro)

import_sub_strategies = [
    "Domestic production capacity can be developed cost-effectively without sacrificing quality",
    "Technology transfer and know-how can be acquired through FDI or licensing arrangements",
    "Import dependency creates strategic vulnerabilities (critical inputs, defense-related items)",
    "Regional content requirements align with GCC common market integration objectives",
    "Local value addition can create employment and enhance industrial capabilities",
    f"{'High-concentration imports (HHI: {:.0f}) present substitution opportunities'.format(hhi_imports) if has_product_data else 'Import concentration analysis identifies priority substitution targets'}",
]

for strategy in import_sub_strategies:
    doc.add_paragraph(strategy, style='List Bullet')

doc.add_heading('4.3 Trade Facilitation and Logistics Optimization' if not has_product_data else '5.3 Trade Facilitation and Logistics Optimization', level=2)

facilitation_intro = f"""
Given the substantial trade volumes (${cum_volume/1e9:.1f}B cumulative) and growth dynamics, both countries should prioritize:
"""
doc.add_paragraph(facilitation_intro)

facilitation_strategies = [
    "Harmonization of customs procedures and documentation requirements to reduce clearance times",
    "Investment in cross-border logistics infrastructure, including cold chain facilities for perishables",
    "Digital trade platforms for streamlined processing, real-time tracking, and electronic documentation",
    "Authorized Economic Operator (AEO) mutual recognition programs for trusted trader benefits",
    "Reduction of non-tariff barriers through regulatory cooperation and technical standards alignment",
    "Single Window implementation for integrated government agency clearances",
    "Risk-based inspection protocols to expedite low-risk shipments while maintaining security",
]

for strategy in facilitation_strategies:
    doc.add_paragraph(strategy, style='List Bullet')

doc.add_page_break()

# ========================================================================
# SECTION 6: IMPLEMENTATION ROADMAP
# ========================================================================
print("  Creating Section 6: Implementation Roadmap...")

doc.add_heading('5. Implementation Roadmap and Action Plan' if not has_product_data else '6. Implementation Roadmap and Action Plan', level=1)

roadmap_intro = f"""
Successful execution of the strategic opportunities requires coordinated action across government agencies, trade promotion bodies,
and private sector stakeholders. The following phased approach prioritizes quick wins while building toward long-term structural transformation.
"""
doc.add_paragraph(roadmap_intro)

doc.add_heading('Phase 1: Immediate Actions (0-6 months)', level=2)

immediate_actions = [
    "Establish bilateral trade committee with quarterly review mechanism and senior-level representation",
    f"Commission detailed market study of high-potential sectors in {PARTNER_COUNTRY}, leveraging product-level trade data",
    "Launch trade mission with B2B matchmaking events to strengthen commercial linkages",
    "Fast-track mutual recognition agreements for product standards, certifications, and testing",
    "Create dedicated export financing facility for SMEs with competitive rates and simplified procedures",
    f"{'Initiate targeted campaigns for Star products identified in BCG analysis' if has_product_data else 'Identify and promote high-growth export products'}",
    "Conduct import substitution feasibility studies for strategic sectors",
]

for i, action in enumerate(immediate_actions, 1):
    doc.add_paragraph(f"{action}", style='List Number')

doc.add_heading('Phase 2: Medium-term Initiatives (6-18 months)', level=2)

medium_actions = [
    "Negotiate preferential market access arrangements for priority products under GCC framework",
    f"Establish permanent trade office in {PARTNER_COUNTRY} or strengthen existing commercial representation",
    f"Launch targeted marketing campaigns {'for Star and Question Mark products' if has_product_data else 'in high-potential product categories'}",
    "Develop supplier capacity building programs to meet export quality standards and certifications",
    "Implement comprehensive trade information portal with real-time market intelligence and logistics tracking",
    "Establish logistics corridor with enhanced infrastructure and streamlined border procedures",
    "Create export consortia for SMEs to achieve economies of scale in marketing and distribution",
    "Initiate technology transfer partnerships in import substitution priority sectors",
]

for i, action in enumerate(medium_actions, 1):
    doc.add_paragraph(f"{action}", style='List Number')

doc.add_heading('Phase 3: Long-term Strategic Initiatives (18+ months)', level=2)

long_actions = [
    "Institutionalize bilateral trade monitoring framework with KPIs and annual assessments",
    "Develop joint industrial cooperation projects in strategic sectors (manufacturing, logistics, services)",
    "Establish integrated logistics and distribution hubs to enhance regional connectivity",
    "Create innovation partnerships and R&D collaborations in high-tech sectors",
    "Implement comprehensive trade facilitation reforms aligned with WTO Trade Facilitation Agreement",
    "Develop free trade zone or special economic zone targeting bilateral trade facilitation",
    f"{'Optimize portfolio mix by divesting from Dog products and reinvesting in Stars' if has_product_data else 'Achieve balanced export portfolio with reduced concentration'}",
    "Achieve sustainable trade balance improvement with coverage ratio target of 75%+",
]

for i, action in enumerate(long_actions, 1):
    doc.add_paragraph(f"{action}", style='List Number')

doc.add_page_break()

# ========================================================================
# SECTION 7: CONCLUSION
# ========================================================================
print("  Creating Section 7: Conclusion...")

doc.add_heading('6. Conclusion and Forward Outlook' if not has_product_data else '7. Conclusion and Forward Outlook', level=1)

conclusion_text = f"""
The bilateral trade relationship between {REPORTING_COUNTRY} and {PARTNER_COUNTRY} presents a complex picture of
deepening economic integration alongside persistent structural imbalances. The analysis reveals:

Strengths:
- Robust trade volume growth ({cagr_volume:.1f}% CAGR) demonstrating strong bilateral economic ties
- {'Dynamic export expansion ({:.1f}% CAGR) showing competitive gains'.format(cagr_exports) if cagr_exports > 0 else 'Export base with potential for acceleration'}
- {'Significant re-export activity ({:.1f}% of total exports) leveraging regional hub positioning'.format(reexport_share) if reexport_share > 15 else 'Foundation for developing regional trade hub capabilities'}
- {'Well-diversified product portfolio (HHI: {:.0f})'.format(hhi_exports) if has_product_data and hhi_exports < 1500 else 'Established trade infrastructure and relationships'}

Challenges:
- Persistent trade deficit (${abs(cum_balance)/1e9:.1f}B cumulative) requiring strategic rebalancing
- {'High' if latest_coverage < 50 else 'Moderate'} import dependency (coverage ratio: {latest_coverage:.1f}%)
- {'Import growth ({:.1f}% CAGR) outpacing exports'.format(cagr_imports) if cagr_imports > cagr_exports else 'Need to sustain export momentum'}
- {'High product concentration (HHI: {:.0f}) creating vulnerability'.format(hhi_exports) if has_product_data and hhi_exports > 2500 else 'Room for export diversification'}

The strategic recommendations and implementation roadmap outlined in this report provide a comprehensive framework for addressing
these challenges while capitalizing on opportunities. Success will require sustained commitment from government agencies, trade
promotion bodies, and the private sector, working in coordination to transform the bilateral trade relationship into a more balanced
and mutually beneficial partnership.

With the GCC economic integration framework providing institutional support, and both countries' Vision 2030+ strategies emphasizing
economic diversification and trade expansion, the conditions are favorable for meaningful progress. The key is translating strategic
intent into concrete actions, with clear accountability, adequate resourcing, and regular monitoring of progress against defined KPIs.
"""
doc.add_paragraph(conclusion_text)

doc.add_paragraph()
doc.add_paragraph()

# Disclaimer
disclaimer = doc.add_paragraph(
    "DISCLAIMER: This report is based on official trade statistics and economic analysis. The strategic recommendations are "
    "provided for informational and planning purposes. Actual policy implementation should be informed by additional stakeholder "
    "consultations, detailed feasibility studies, cost-benefit analysis, and consideration of broader economic and political factors. "
    "The authors and Deloitte Consulting assume no liability for decisions made based on this analysis."
)
disclaimer.runs[0].font.size = Pt(9)
disclaimer.runs[0].font.italic = True
disclaimer.runs[0].font.color.rgb = RGBColor(100, 100, 100)

# ========================================================================
# SAVE DOCUMENT
# ========================================================================
print("\n  Saving document...")

output_filename = f'{REPORTING_COUNTRY}_{PARTNER_COUNTRY}_Comprehensive_Trade_Report_{datetime.now().strftime("%Y%m%d")}.docx'
doc.save(f'/content/{output_filename}')

print(f"\n{'='*80}")
print(f"✅ COMPREHENSIVE REPORT GENERATED SUCCESSFULLY!")
print(f"{'='*80}")
print(f"\n📄 Filename: {output_filename}")
print(f"📊 Sections: {6 if not has_product_data else 7}")
print(f"📈 Figures: {7 if has_product_data else 5}")
print(f"📋 Tables: {8 if has_product_data else 4}")
print(f"📄 Estimated Pages: {'20-25' if has_product_data else '15-18'}")
print(f"\nReport includes:")
print(f"  ✓ Executive Summary with 13+ KPIs")
print(f"  ✓ Trade Balance Analysis (annual table)")
print(f"  ✓ Growth Dynamics (CAGR, volatility, trends)")
print(f"  ✓ Coverage Analysis (import dependency)")
print(f"  ✓ Re-export Dynamics")
if has_product_data:
    print(f"  ✓ BCG Portfolio Matrix")
    print(f"  ✓ HHI Concentration Analysis")
    print(f"  ✓ Top Products Tables")
print(f"  ✓ Strategic Recommendations (15+ actions)")
print(f"  ✓ 3-Phase Implementation Roadmap")
print(f"  ✓ 7 Professional Visualizations")
print(f"\n{'='*80}")


📄 GENERATING COMPREHENSIVE WORD DOCUMENT REPORT
  Creating title page...
  Creating executive summary...
  Creating Section 1: Trade Balance Analysis...
  Creating Section 2: Growth Dynamics...
  Creating Section 3: Trade Balance Decomposition...
  Creating Section 5: Strategic Recommendations...
  Creating Section 6: Implementation Roadmap...
  Creating Section 7: Conclusion...

  Saving document...

✅ COMPREHENSIVE REPORT GENERATED SUCCESSFULLY!

📄 Filename: Oman_KSA_Comprehensive_Trade_Report_20251208.docx
📊 Sections: 7
📈 Figures: 7
📋 Tables: 8
📄 Estimated Pages: 20-25

Report includes:
  ✓ Executive Summary with 13+ KPIs
  ✓ Trade Balance Analysis (annual table)
  ✓ Growth Dynamics (CAGR, volatility, trends)
  ✓ Coverage Analysis (import dependency)
  ✓ Re-export Dynamics
  ✓ BCG Portfolio Matrix
  ✓ HHI Concentration Analysis
  ✓ Top Products Tables
  ✓ Strategic Recommendations (15+ actions)
  ✓ 3-Phase Implementation Roadmap
  ✓ 7 Professional Visualizations



In [52]:
# ============================================================================
# CELL 11: DOWNLOAD THE COMPREHENSIVE REPORT
# ============================================================================

from google.colab import files
import os

print("\n" + "="*80)
print("📥 DOWNLOADING YOUR COMPREHENSIVE REPORT")
print("="*80)

# List all generated files
print("\n📁 Generated files:")
for f in os.listdir('/content'):
    if f.endswith('.docx') and 'Comprehensive_Trade_Report' in f:
        size = os.path.getsize(f'/content/{f}')
        print(f"  ✓ {f} ({size:,} bytes)")

# Download the report
if os.path.exists(f'/content/{output_filename}'):
    print(f"\n🔽 Downloading: {output_filename}")
    files.download(f'/content/{output_filename}')
    print("\n✅ DOWNLOAD COMPLETE!")
    print("\nYour comprehensive Word document is now saved to your PC!")
    print("\n" + "="*80)
    print("🎉 ANALYSIS COMPLETE - ENJOY YOUR REPORT!")
    print("="*80)
else:
    print("\n❌ Report file not found!")

print("\n📊 REPORT CONTENTS:")
print("   • Executive Summary with complete KPI table")
print("   • Trade Balance Analysis with annual data table")
print("   • Growth Dynamics with CAGR charts")
print("   • Coverage Analysis with trend visualization")
print("   • Volatility Assessment")
print("   • Re-export Dynamics")
if has_product_data:
    print("   • BCG Portfolio Matrix with product classification")
    print("   • HHI Concentration Analysis")
    print("   • Top Products Tables (Stars, Top 10 exports/imports)")
print("   • Strategic Recommendations (15+ actionable strategies)")
print("   • 3-Phase Implementation Roadmap (20+ specific actions)")
print("   • Professional charts embedded throughout")
print("\n" + "="*80)


📥 DOWNLOADING YOUR COMPREHENSIVE REPORT

📁 Generated files:
  ✓ Oman_KSA_Comprehensive_Trade_Report_20251208.docx (1,457,020 bytes)

🔽 Downloading: Oman_KSA_Comprehensive_Trade_Report_20251208.docx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ DOWNLOAD COMPLETE!

Your comprehensive Word document is now saved to your PC!

🎉 ANALYSIS COMPLETE - ENJOY YOUR REPORT!

📊 REPORT CONTENTS:
   • Executive Summary with complete KPI table
   • Trade Balance Analysis with annual data table
   • Growth Dynamics with CAGR charts
   • Coverage Analysis with trend visualization
   • Volatility Assessment
   • Re-export Dynamics
   • BCG Portfolio Matrix with product classification
   • HHI Concentration Analysis
   • Top Products Tables (Stars, Top 10 exports/imports)
   • Strategic Recommendations (15+ actionable strategies)
   • 3-Phase Implementation Roadmap (20+ specific actions)
   • Professional charts embedded throughout

